In [ ]:
!pip install shapely==2.0.1 -q
!pip install segmentation_models_pytorch -q

In [ ]:
import bz2
import lzma
import pickle

import matplotlib.pyplot as plt
import numpy as np
import shapely
import tqdm
from shapely.geometry import box
from shapely.geometry import MultiPolygon, Point, GeometryCollection, MultiLineString
from shapely import affinity
from uuid import uuid4
import geopandas as gpd
import bz2
import lzma
import pickle
import matplotlib.pyplot as plt
import numpy as np
import shapely
import tqdm
from shapely.geometry import box
from shapely.geometry import MultiPolygon, Point, GeometryCollection, MultiLineString
from shapely import affinity
from uuid import uuid4
from typing import Iterable
import random
import cv2, pickle
import numpy as np
import shapely
from pandas import Series
from shapely import MultiPolygon, Polygon, box, Point
from shapely.affinity import scale
import gc, torch


In [ ]:
c = 0
size = 256

def buffer(p, w, bm=0.5):
    p = affinity.scale(p, xfact=0.9, yfact=0.9, origin=(size / 2, size / 2)) if p else Point(-100, -100)
    if bm:
        if isinstance(p, shapely.geometry.multipolygon.MultiPolygon):
            return [i.buffer(-bm * w, join_style=2).buffer((1.5 + bm) * w, join_style=2).buffer(-1.5 * w, join_style=2)
                    for i in p.geoms]
        return [p.buffer(-bm * w, join_style=2).buffer((1.5 + bm) * w, join_style=2).buffer(-1.5 * w, join_style=2)]
    return p


In [ ]:
plans = pickle.load(open('/kaggle/input/with-kit/Planify_ocr_kit.pkl', 'rb'))
old_plans = pickle.load(open('/kaggle/input/planify-cleaned-neigh/Planify_cleaned_neigh.pkl', 'rb'))

In [ ]:
for i, plan in enumerate(plans):
    old_plan = old_plans[i]
    plan["neigh"] = old_plan["neigh"]

In [ ]:
def get_plan_width(plan):
    inner = plan["inner"]
    x1, y1, x2, y2 = inner.bounds
    w, h = x2-x1, y2-y1
    diameter = max(w, h)
    return diameter

plans = [i for i in plans if get_plan_width(i) <= 260]

In [ ]:
aug_prefs = [
    [0, False],
    [0, True],
    [90, False],
    [90, True],
    [180, False],
    [180, True],
    [270, False],
    [270, True]
]

def augment(polygon, degree, flip_vertical, scale=1, size=256, point=False):
    if not polygon:
        return Point(-100, -100) if point else box(-100, -100, -100, -100)
    p = affinity.rotate(polygon, degree, origin=(size / 2, size / 2))
    flip = 1
    if flip_vertical:
        flip = -1
    p = affinity.scale(p, xfact=scale, yfact=scale * flip, origin=(size / 2, size / 2))
    return p

In [ ]:

def get_mask(poly, shape=(256, 256), point_s=5, line_s=0):
    """ Return image contains multiploygon as a numpy array mask

    Parameters
    ----------
    poly: Polygon or MultiPolygon or Iterable[Polygon or MultiPolygon]
        The Polygon/s to get mask for
    shape: tuple
        The shape of the canvas to draw polygon/s on

    Returns
    -------
    ndarray
        Mask array of the input polygon/s
        :param point_s:

    """

    img = np.zeros(shape, dtype=np.uint8)
    if isinstance(poly, Polygon):
        if poly.is_empty:
            return img
        if line_s:
            points = np.array(poly.exterior.coords[:], dtype=np.int32)
            img = cv2.polylines(img, [points], True, 255, thickness=line_s)
        else:
            img = cv2.drawContours(img, np.int32([poly.exterior.coords]), -1, 255, -1)

        for interior in poly.interiors:

            points = np.array(interior.coords[:], dtype=np.int32)
            if line_s:
                img = cv2.polylines(img, [points], True, 0, thickness=line_s)
            else:
                img = cv2.drawContours(img, np.int32([interior.coords]), -1, 0, -1)

    elif isinstance(poly, MultiPolygon):
        for p in poly.geoms:
            if line_s:
                points = np.array(p.exterior.coords[:], dtype=np.int32)
                img = cv2.polylines(img, [points], True, 255, thickness=line_s)
            else:
                img = cv2.drawContours(img, np.int32([p.exterior.coords]), -1, 255, -1)

    elif isinstance(poly, Series):
        polys = [p for p in poly.tolist() if p]
        img = get_mask(polys, shape, point_s, line_s)

    elif isinstance(poly, Iterable):
        for p in poly:
            img = (img != 0) | (get_mask(p, shape, point_s, line_s) != 0)
        img = img.astype(np.uint8) * 255
    elif isinstance(poly, Point):
        p = poly.coords[0]
        img = cv2.circle(img, (int(p[0]), int(p[1])), point_s, 255, -1)
    return img.astype(np.uint8)


In [ ]:
def get_centroid(poly):
    x1, x2, y1, y2 = 0, 0, 0, 0

In [ ]:

MIN_ROOM_AREA = 8.5
MAX_ROOM_AREA = 46
MIN_BROOM_AREA = 3.5
MAX_BROOM_AREA = 15

def rand_room_area(n):
    total = 0
    for i in range(n):
        MIDPOINT = (MIN_ROOM_AREA + MAX_ROOM_AREA) / 2
        STD_DEV = (MAX_ROOM_AREA - MIN_ROOM_AREA) / 4
        room_area = random.normalvariate(MIDPOINT, STD_DEV)
        room_area = max(min(room_area, MAX_ROOM_AREA), MIN_ROOM_AREA)
        room_area = round(room_area, 2)
        total += room_area
    return total

def rand_broom_area(n):
    total = 0
    for i in range(n):
        MIDPOINT = (MIN_BROOM_AREA + MAX_BROOM_AREA) / 2
        STD_DEV = (MAX_BROOM_AREA - MIN_BROOM_AREA) / 4
        room_area = random.normalvariate(MIDPOINT, STD_DEV)
        room_area = max(min(room_area, MAX_BROOM_AREA), MIN_BROOM_AREA)
        room_area = round(room_area, 2)
        total += room_area
    return total

def estimate_total_area(num_bedrooms, num_bathrooms):
    avg_ratio_1br_1ba = 0.6
    avg_ratio_2br_2ba = 0.7
    avg_ratio_3br_nba = 0.75
    combined_bed_bath_area = rand_room_area(num_bedrooms) + rand_broom_area(num_bathrooms)
    if num_bedrooms == 1 and num_bathrooms == 1:
        area_ratio = avg_ratio_1br_1ba
    elif num_bedrooms == 2 and num_bathrooms == 2:
        area_ratio = avg_ratio_2br_2ba
    else:
        area_ratio = avg_ratio_3br_nba
    total_area = combined_bed_bath_area / area_ratio

    return total_area



In [ ]:
import gc, torch
torch.cuda.empty_cache()
gc.collect()

In [ ]:
def perturb_polygon(polygon, x_range=(-2, 2), y_range=(-2, 2)):
    """
    Apply random perturbation to the coordinates of a shapely polygon.

    Args:
        polygon (Polygon): The original shapely polygon.
        x_range (tuple): The range for random perturbation in x-axis. Default is (-0.1, 0.1).
        y_range (tuple): The range for random perturbation in y-axis. Default is (-0.1, 0.1).

    Returns:
        Polygon: The perturbed shapely polygon.
    """
    # Get the coordinates of the original polygon
    original_coords = np.array(polygon.exterior.coords)

    # Iterate through each vertex of the original polygon
    perturbed_coords = []
    for x, y in original_coords:
        # Generate a random perturbation for x and y coordinates within the specified range
        perturbation_x = np.random.uniform(x_range[0], x_range[1])
        perturbation_y = np.random.uniform(y_range[0], y_range[1])

        # Apply the perturbation to the original coordinates
        perturbed_x = x + perturbation_x
        perturbed_y = y + perturbation_y

        perturbed_coords.append((perturbed_x, perturbed_y))

    # Create a new Polygon object with the perturbed coordinates
    perturbed_polygon = Polygon(perturbed_coords)

    return perturbed_polygon


In [ ]:

def noise(point, noise_scale=10):
    # Add random noise to the point
    x, y = point.coords[0]
    dx_noise = random.uniform(-noise_scale, noise_scale)
    dy_noise = random.uniform(-noise_scale, noise_scale)
    noisy_point = Point(x + dx_noise, y + dy_noise)


    return noisy_point

In [ ]:
# plan = plans[10]
# inner = perturb_polygon(plan['inner'].geoms[0]).buffer(0)
# front = plan['front'].buffer(4).intersection(inner.exterior).centroid
# gpd.GeoSeries([front, inner]).plot(cmap='Dark2_r')
# print(front,  plan['front'])

In [ ]:
len([i for i in plans if len(i['bathroom'].geoms) == 4 ])

# Dataset

In [ ]:
import numpy as np
import keras, os, random
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

onehot = {}
for i in [1, 2, 3, 4]:
    for j in [1, 2, 3, 4]:
        img = np.zeros((256, 256, 8))
        img[:, :, i-1] =  np.ones((256, 256))
        img[:, :, j+4-1] = 1
        onehot[(i, j)] = img[:, :, :]

import torch
import numpy as np
from torch.utils.data import Dataset

import torch
from torch.utils.data import Dataset

def centroid(poly):
    x1, y1, x2, y2 = poly.bounds
    return Point((x2+x1)/2, (y2+y1)/2)

class PlanDataset(Dataset):
    def __init__(self, plans, batch_size=32, image_size=(256, 256), aug_p=None):
        self.plans = plans
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_samples = len(self.plans)
        self.aug_p = aug_p

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        plan = self.plans[idx]
        deg, flip = random.choice(aug_prefs)
        deg = deg + random.randint(-3, 3)
        
        if self.aug_p:
            deg, flip = self.aug_p
#         deg = deg + random.randint(-5, 5)
        no_bedrooms = min(max(1, len(plan['bedroom'].geoms)), 3)
        
        no_bathrooms = min(max(1, len(plan['bathroom'].geoms)), 3)
        
#         area = estimate_total_area(no_bedrooms, no_bathrooms)

#         case = random.choice([f for f in range(1, no_bedrooms+1) for j in range(f)]) if no_bedrooms > 1 else 1
#         inbedsnum = case-1

        x = np.zeros((*self.image_size, 2))
        x1 = np.zeros((*self.image_size, 3))
        y = np.zeros((*self.image_size, 1))
        
#         deg = random.randint(0, 360)

#         x[:,:,:8] = onehot[(no_bedrooms, no_bathrooms)][:,:,:8]
#         x[:,:,9] = get_mask(augment(plan['front'], deg, flip), (256, 256), point_s=7) > 0
#         x[:,:,10] = get_mask(augment(plan['inner_g'], deg, flip), (256, 256), point_s=10) > 0
#         x[:,:,11 + inbedsnum] = 1
        s = 0.8
        inner = perturb_polygon(plan['inner'].geoms[0] , x_range=(-0, 0), y_range=(-0, 0)).buffer(0)
#         inner = plan['inner'].buffer(0)
    
        noif = 2

        inner = inner if isinstance(inner, Polygon) else max(inner.geoms, key=lambda x:x.area)
        front = plan['front'].buffer(8).intersection(inner.exterior).centroid
        if not front:
            front = plan['front']
        front = augment(front, deg, flip, s)
        
#         bedrooms = sorted([augment(f, deg, flip, s) for f in plan['bedroom'] if inner.contains(f)], key=lambda pp:pp.area, reverse=True)
#         bathrooms = sorted([augment(f, deg, flip, s) for f in plan['bathroom'] if inner.contains(f)], key=lambda pp:pp.area, reverse=True)
#         inner_ext = augment(inner, deg, flip, s).exterior.buffer(10)
#         bedrooms = sorted([augment(f, deg, flip, s) for f in plan['bedroom'] if inner.contains(f)], key=lambda pp:pp.exterior.buffer(5).intersection(inner_ext).area, reverse=True)
#         bathrooms = sorted([augment(f, deg, flip, s) for f in plan['bathroom'] if inner.contains(f)], key=lambda pp:pp.exterior.buffer(5).intersection(inner_ext).area, reverse=True)

        try:
            bedrooms = sorted([augment(f, deg, flip, s) for f in plan['bedroom'].geoms], key=lambda pp:pp.centroid.distance(front), reverse=True)
            bathrooms = sorted([augment(f, deg, flip, s) for f in plan['bathroom'].geoms], key=lambda pp:pp.centroid.distance(front), reverse=True)
        except:
            print(front, [augment(f, deg, flip, s) for f in plan['bedroom'].geoms], [augment(f, deg, flip, s) for f in plan['bathroom'].geoms])
#         bedroomscn = [noise(f.centroid, noif) for f in bedrooms]
#         bathroomscn = [noise(f.centroid, noif) for f in bathrooms]
        bedroomsc = [f.centroid for f in bedrooms]
        bathroomsc = [f.centroid for f in bathrooms]
        x[:,:,0] = get_mask(front, (256, 256), point_s=7) > 0
        x[:,:,1] = get_mask(augment(inner, deg, flip, s), (256, 256), point_s=10) > 0
#         x[:,:,2] = get_mask(bedroomsc, (256, 256), point_s=5) > 0
#         x[:,:,3] = get_mask(bathroomsc, (256, 256), point_s=5) >0
#         x[:,:,2:] = get_mask(bedroomsc, (256, 256), point_s=5) > 0
#         bedrooms = [f.buffer(0, join_style=2, cap_style=2) for f in bedrooms]
#         bathrooms = [f.buffer(0, join_style=2, cap_style=2) for f in bathrooms]
#         x[:,:,2] = min(1, area / 500)


#         x[:,:,2:10] = onehot[(no_bedrooms, no_bathrooms)]


#         random.shuffle(bedroomsc)
        
        
#         x[:,:,10] = get_mask([noise(bp, noif) for bp in bedroomsc], (256, 256), point_s=8) > 0
#         x[:,:,11] = get_mask([noise(bp, noif) for bp in bathroomsc], (256, 256), point_s=8) > 0

#         kitchen = augment(plan['kitchen'], deg, flip,  s).centroid
        
#         x[:,:,3] = get_mask(bedrooms, (256, 256), point_s=5) > 0
        

#         inbeds = []
#         for _ in range(inbedsnum):
#             bedroom = random.choice(bedrooms)
#             bedc = bedroom.minimum_rotated_rectangle.centroid if bedroom else Point(-100, -100)
#             bedrooms = [x for x in bedrooms if x != bedroom]
#             inbeds.append(augment(bedc, deg, flip))
#         general = MultiPolygon(plan['general']) if isinstance(plan['general'], list) else plan['general']
#         general = inner if isinstance(general, Polygon) else max(general.geoms, key=lambda x:x.area)
#         x[:,:,10] = get_mask(bedroomsc[0], (256, 256), point_s=8) > 0
        y[:,:,0] = get_mask(augment(plan['kitchen'].centroid, deg, flip, s), (256, 256), point_s=8) > 0
        x1[:,:,0] = get_mask(bedroomsc, (256, 256), point_s=8) > 0
        x1[:,:,1] = get_mask(bathroomsc, (256, 256), point_s=8) > 0
#         y[:,:,1] = get_mask(bathroomsc[:1], (256, 256), point_s=6) > 0
#         y[:,:,2] = get_mask(augment(general, deg, flip, scale=s).centroid, point_s=8) > 0

#         bedroom = random.choice(bedrooms).minimum_rotated_rectangle.centroid if bedrooms else Point(-100, -100)
#         walls = augment(plan['wall'].buffer(2, join_style=2, cap_style=2), deg, flip, s * 0.9)
#         doors = augment(plan['door'].buffer(2, join_style=2, cap_style=2), deg, flip, s * 0.9)
#         windows = augment(plan['window'].buffer(2, join_style=2, cap_style=2), deg, flip, s * 0.9)
#         y[:,:,0] = (get_mask(walls, (256, 256), point_s=5)/255.0 + get_mask(doors, (256, 256), point_s=5)/255.0 + get_mask(windows, (256, 256), point_s=5)/255.0)>0
#         y[:,:,1] = (get_mask(walls, (256, 256), point_s=5)/255.0 + get_mask(doors, (256, 256), point_s=5)/255.0 + get_mask(windows, (256, 256), point_s=5)/255.0)>0
#######################################################################
#         y[:,:,0] = (get_mask(walls, (256, 256), point_s=5)/255.0 + get_mask(doors, (256, 256), point_s=5)/255.0 + get_mask(windows, (256, 256), point_s=5)/255.0)>0
#         y[:,:,0] *= 1-y[:,:,2]
#         y[:,:,1] *= 1-y[:,:,2]
#######################################################################
#         y[:,:,1] = get_mask([d.centroid for d in doors.geoms]  if isinstance(doors, MultiPolygon) else doors.centroid, (256, 256), point_s=3) > 0
#         y[:,:,2] = get_mask([d.centroid for d in windows.geoms] if isinstance(windows, MultiPolygon) else windows.centroid, (256, 256), point_s=3) > 0
#         all_romes = shapely.ops.unary_union(bedrooms+bathrooms )
#         buffered = [br.buffer(plan['wall_width'], join_style=2).difference(all_romes) for br in bedrooms + bathrooms ]
#         y[:,:,0] = get_mask(buffered, (256, 256), point_s=7) > 0
#         y[:,:,2] = get_mask(kitchen, (256, 256), point_s=7) > 0
        area = plan['area']
        area += random.randint(-5, 20)
        
        one_hot = torch.zeros((4, 8, 8)).float()
        one_hot[no_bedrooms-1] = 1.0
        return torch.from_numpy(x).permute(2, 0, 1).float(), torch.tensor(norm_area(area)).float(), torch.from_numpy(y).permute(2, 0, 1).float(), one_hot.float(), torch.from_numpy(x1).permute(2, 0, 1).float()




In [ ]:
nbedd = {f"bed{i+1}":i for i in range(3)}
nbathd = {f"bath{i+1-3}":i for i in range(3, 6)}
nd = {**nbedd, **nbathd}

In [ ]:
def get_width_height(poly):
    x1, y1, x2, y2 = poly.bounds
    return x2-x1, y2-y1

In [ ]:
from shapely.ops import unary_union
import numpy as np
import os, random
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch.utils.data import Dataset

import torch
from torch.utils.data import Dataset

In [ ]:
def get_centroids(list_rooms):
    return [centroid(r) for r in list_rooms]

In [ ]:
onehot.keys()

In [ ]:
def norm_area(area):
    return (area - 30) / 220

def restore_area(area):
    return area * 220 + 30


onehot = {}
for i in [1, 2, 3, 4]:
    for j in [1, 2, 3, 4]:
        img = torch.zeros((256, 256, 8))
        img[:, :, i-1] =  torch.ones((256, 256))
        img[:, :, j+4-1] = 1
        onehot[(i, j)] = img[:, :, :]



class PlanDataset(Dataset):
    def __init__(self, plans, batch_size=32, image_size=(256, 256), aug_p=None, state=None, val=False):
        self.plans = plans
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_samples = len(self.plans)
        self.aug_p = aug_p
        self.val=val
        self.state = state
        
    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        plan = self.plans[idx]
        deg, flip = random.choice(aug_prefs)
#         deg = deg + random.randint(-3, 3)
        s = 0.8
        buffer_range = 0
        
        area = min(plan['size'] / 400, 1)
        
        

        if self.aug_p:
            deg, flip = self.aug_p
            
        perturb_range = 4
        
        if not self.val:
            x1, y1, x2, y2 = plan['inner'].bounds
            w, h = x2-x1, y2-y1
            
            mi, ma = min(w, h), max(w, h)
            buffer_range = random.randint(0, 30)
            buffer_range = buffer_range * mi / ma
#             deg = deg + random.randint(-5, 5)
#             perturb_range = 3
            perturb_range = random.randint(0, perturb_range)
            perturb_range = 0
            
        ww = plan["wall_width"]
        no_bedrooms = min(max(1, len(plan['bedroom'].geoms)), 4)
        no_bathrooms = min(max(1, len(plan['bathroom'].geoms)), 4)
        
        neighbours, non_neighbours = plan["neigh"]
        
        neighbours = augment(unary_union(neighbours), deg, flip, s)
        non_neighbours = augment(unary_union(non_neighbours), deg, flip, s).buffer(ww)
        
        
        
        x1 = np.zeros((*self.image_size, 12))
        x2 = np.zeros((*self.image_size, 3))
        y = np.zeros((*self.image_size, 1))
        
        inner = plan['inner'].geoms[0]
        inner = inner.buffer(buffer_range, join_style=2, cap_style=2).buffer(-buffer_range, join_style=2, cap_style=2)
        
#         try:
#             inner = perturb_polygon( inner, x_range=(-perturb_range, perturb_range), y_range=(-perturb_range, perturb_range)).buffer(0)
#         except:
#             ...
    
        inner = inner if isinstance(inner, Polygon) else max(inner.geoms, key=lambda x:x.area)
        front = plan['front_door'].buffer(8).intersection(inner.exterior)
        random_distance = random.uniform(0, front.length)
        front = front.interpolate(random_distance)
        
        if not front:
            front = plan['front_door']
            
        front = augment(front, deg, flip, s)
        inner = augment(inner, deg, flip, s)

        bedrooms = [augment(f, deg, flip, s) for f in plan['bedroom'].geoms]
        bathrooms = [augment(f, deg, flip, s) for f in plan['bathroom'].geoms]
        kitchen = [augment(f, deg, flip, s) for f in plan['kitchen'].geoms]
        blacony = [augment(f, deg, flip, s) for f in plan['balacony'].geoms]
        door = [augment(f, deg, flip, s) for f in plan['door'].geoms]
        window = [augment(f, deg, flip, s) for f in plan['window'].geoms]
        

        if self.state is None:
            state = random.choice([4, 5, 6])
        else:
            state = self.state
        
#         state = 1
#         state=4

        x1[:,:,0] = get_mask(front, (256, 256), point_s=5) > 0
        x1[:,:,1] = get_mask(inner, (256, 256), point_s=5) > 0
        length_box = 256*area/2
        x1[:,:,2] = get_mask(box(128-length_box, 128-length_box, 128+length_box, 128+length_box), (256, 256), point_s=10) > 0        
        x1[:,:,3] = get_mask(neighbours, (256, 256), point_s=5) > 0
        
        if state == 1:
            buf_nei = neighbours.buffer(ww*4)
            bedrooms = sorted(bedrooms, key=lambda bedr: bedr.intersection(buf_nei).length, reverse=False)

        
            bedroomsc = [centroid(f) for f in bedrooms]
            bathroomsc = [centroid(f) for f in bathrooms]
            kitchenc = [f.centroid for f in kitchen]

            out_idx = random.choice(range(len(bedroomsc)))

            y[:,:,0] = get_mask(bedroomsc[out_idx], (256, 256), point_s=5) > 0

            bedroomsc.pop(out_idx)
            bedroomsc = bedroomsc[:out_idx]

            if not self.val:
                bedroomsc = [noise(f, 2) for f in bedroomsc]        


            x2[:,:,0] = get_mask(bedroomsc, (256, 256), point_s=5) > 0
#             x2[:,:,3] = 1.0
        
        
        elif state == 2:
            un_bed = unary_union(bedrooms).buffer(ww*2)
            bathrooms = sorted(bathrooms, key=lambda bedr: bedr.intersection(un_bed).length, reverse=False)
            
            

            bedroomsc = [centroid(f) for f in bedrooms]
            bathroomsc = [centroid(f) for f in bathrooms]
            kitchenc = [f.centroid for f in kitchen]
        
            out_idx = random.choice(range(len(bathroomsc)))

            y[:,:,0] = get_mask(bathroomsc[out_idx], (256, 256), point_s=5) > 0

            bathroomsc.pop(out_idx)
            bathroomsc = bathroomsc[:out_idx]

            if not self.val:
                bathroomsc = [noise(f, 2) for f in bathroomsc]        
                bedroomsc = [noise(f, 2) for f in bedroomsc]        

            x2[:,:,0] = get_mask(bedroomsc, (256, 256), point_s=5) > 0
            x2[:,:,1] = get_mask(bathroomsc, (256, 256), point_s=5) > 0
#             x2[:,:,4] = 1.0

        elif state == 3:
    
            bedroomsc = [centroid(f) for f in bedrooms]
            bathroomsc = [centroid(f) for f in bathrooms]
            kitchenc = [f.centroid for f in kitchen]
            
            if not self.val:
                bathroomsc = [noise(f, 2) for f in bathroomsc]        
                bedroomsc = [noise(f, 2) for f in bedroomsc]   
            
            y[:,:,0] = get_mask(kitchenc, (256, 256), point_s=5) > 0
            x2[:,:,0] = get_mask(bedroomsc, (256, 256), point_s=5) > 0
            x2[:,:,1] = get_mask(bathroomsc, (256, 256), point_s=5) > 0

        elif state == 4:
    
            x2[:,:,0] = get_mask(bedrooms, (256, 256), point_s=5) > 0
            x2[:,:,1] = get_mask(bathrooms, (256, 256), point_s=5) > 0
            x2[:,:,2] = get_mask(kitchen, (256, 256), point_s=5) > 0
            y[:,:,0] = get_mask(get_centroids(blacony), (256, 256), point_s=5) > 0
            
        elif state == 5:
    
            x2[:,:,0] = get_mask(bedrooms, (256, 256), point_s=5) > 0
            x2[:,:,1] = get_mask(bathrooms, (256, 256), point_s=5) > 0
            x2[:,:,2] = get_mask(kitchen, (256, 256), point_s=5) > 0
            y[:,:,0] = get_mask(get_centroids(door), (256, 256), point_s=5) > 0
            
        elif state == 6:
    
            x2[:,:,0] = get_mask(bedrooms, (256, 256), point_s=5) > 0
            x2[:,:,1] = get_mask(bathrooms, (256, 256), point_s=5) > 0
            x2[:,:,2] = get_mask(kitchen, (256, 256), point_s=5) > 0
            y[:,:,0] = get_mask(get_centroids(window), (256, 256), point_s=5) > 0
            
        x1 = torch.from_numpy(x1)
        x1[:,:,4:] = onehot[(no_bedrooms, no_bathrooms)]
        x1 = x1.permute(2, 0, 1).float()
        
        return x1, torch.from_numpy(x2).permute(2, 0, 1).float(), torch.from_numpy(y).permute(2, 0, 1).float(), state




In [ ]:

def plot_eight(images):
    width = 256
    height = 256
    rows = 2
    cols = 4
    fig = plt.figure(figsize=(12, 8))  # Set the width and height as desired
    fig.suptitle("Polygon Perturbation + Transformation + Rotational Noise", fontsize=16)  # Add the title to the whole plot
    axes = []


    for i, image in enumerate(images):
        image = cv2.resize((image.numpy() * 255).astype(np.uint8), (512, 512))
        a, b = aug_prefs[i]
#         a = 0
#         b = 0
        axes.append(fig.add_subplot(rows, cols, i + 1))
        axes[-1].set_title("Rotation: " + str(a) + " | Flip X: " + ("True" if b else "False"))
        axes[-1]
        plt.imshow(image)
#     
    fig.tight_layout()
    plt.figure(figsize=(30, 30))

    plt.show()

In [ ]:
itt = iter(PlanDataset([plans[7]]))
r = next(iter(itt))
plt.imshow(r[0].permute((1, 2, 0))[:, :, [1, 2, 3]])
plt.show()
plt.imshow(r[2].permute((1, 2, 0))[:, :, 0])
plt.show()
plt.imshow(r[1].permute((1, 2, 0))[:, :, 0])
plt.show()

# Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from segmentation_models_pytorch import Unet
from segmentation_models_pytorch.encoders import get_encoder
from segmentation_models_pytorch.decoders.unet.decoder import UnetDecoder
from segmentation_models_pytorch.base import SegmentationModel, SegmentationHead
from segmentation_models_pytorch.base.modules import Activation
from segmentation_models_pytorch.base import Conv2dReLU
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
from efficientnet_pytorch.utils import Conv2dStaticSamePadding
import torch
from torchvision import models
from torch import nn
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torch import nn
import cv2
import pytorch_lightning as pl
import torch
from sklearn.model_selection import train_test_split
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:


class CustomUnetPlusPlus(smp.UnetPlusPlus):
    def __init__(self, encoder_name='efficientnet-b7', encoder_weights='imagenet', in_channels=20, classes=1):
        super().__init__(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes)
        

# model_2 = CustomUnetPlusPlus().cuda()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class DeepDecoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(2560, 1280, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(1280, 720, kernel_size=3, stride=2, padding=1)
        self.convTrans1 = nn.ConvTranspose2d(720, 640, kernel_size=2, stride=2)
        self.convTrans2 = nn.ConvTranspose2d(640, 320, kernel_size=2, stride=2)
        self.convTrans3 = nn.ConvTranspose2d(320, 160, kernel_size=2, stride=2)
        self.convTrans4 = nn.ConvTranspose2d(160, 80, kernel_size=2, stride=2)
        self.convTrans5 = nn.ConvTranspose2d(80, 1, kernel_size=2, stride=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.convTrans1(x))
        x = F.relu(self.convTrans2(x))
        x = F.relu(self.convTrans3(x))
        x = F.relu(self.convTrans4(x))
        x = self.convTrans5(x)  # No activation on final layer
        return x



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, inch=3, outch=512):
        super().__init__()
        self.conv1 = nn.Conv2d(inch, 64, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(512, outch, kernel_size=3, stride=1, padding=1)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        return x
    
    
class DecoderCNN2(nn.Module):
    def __init__(self, inch=2560, outch=1):
        super(DecoderCNN2, self).__init__()
        
        self.conv1 = nn.Conv2d(2560, 512, kernel_size=3, padding=1) 
        self.bn1 = nn.BatchNorm2d(512)
        self.conv2 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)  
        self.conv4 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 1, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))) 
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.conv5(x)
        return torch.sigmoid(F.interpolate(x, size=(256, 256)))


In [ ]:
nn.Conv2d(32, 1, kernel_size=3, padding=1)(torch.zeros((1, 32 , 32, 32))).shape

In [ ]:
import torch
import torch.nn as nn
from torch.nn import LSTM
import torchvision

import torch
import torch.nn as nn

class DecoderCNN(nn.Module):
    def __init__(self):
       super().__init__()
        
       self.conv1 = nn.Conv2d(2560, 2048, kernel_size=3, padding=1)
       self.bn1 = nn.BatchNorm2d(2048)
    

       self.deconv1 = nn.ConvTranspose2d(2048, 512, kernel_size=4, stride=2, padding=1, output_padding=0)
       self.bn3 = nn.BatchNorm2d(512)

       self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, output_padding=0)  
       self.bn4 = nn.BatchNorm2d(256)
       
       self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, output_padding=0)
       self.bn5 = nn.BatchNorm2d(128)

       self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
       self.conv4 = nn.Conv2d(64, 16, kernel_size=3, padding=1)
       self.conv5 = nn.Conv2d(16, 1, kernel_size=3, padding=1)

    def forward(self, x):
       # Encoder 
       x = F.relu(self.bn1(self.conv1(x)))         
       x = F.relu(self.bn3(self.deconv1(x)))
       x = F.relu(self.bn4(self.deconv2(x)))  
       x = F.relu(self.bn5(self.deconv3(x)))
       x =  F.relu(self.conv3(x))
       x =  F.relu(self.conv4(x))
       x = torch.sigmoid(self.conv5(x))
        
       return x


class DecoderCNN(nn.Module):
    def __init__(self):
       super().__init__()
        
       self.conv1 = nn.Conv2d(2560, 2048, kernel_size=3, padding=1)
       self.bn1 = nn.BatchNorm2d(2048)
    

       self.deconv1 = nn.ConvTranspose2d(2048, 64, kernel_size=4, stride=2, padding=1, output_padding=0)
       self.bn3 = nn.BatchNorm2d(64)

       self.deconv2 = nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1, output_padding=0)  
       self.bn4 = nn.BatchNorm2d(32)
       
       self.deconv3 = nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1, output_padding=0)
       self.bn5 = nn.BatchNorm2d(16)

       self.conv3 = nn.Conv2d(16, 1, kernel_size=3, padding=1)

    def forward(self, x):
       # Encoder 
       x = F.relu(self.bn1(self.conv1(x)))         
       x = F.relu(self.bn3(self.deconv1(x)))
       x = F.relu(self.bn4(self.deconv2(x)))  
       x = F.relu(self.bn5(self.deconv3(x)))
       x = torch.sigmoid(self.conv3(x))
        
       return x

class BedCentRegressor(pl.LightningModule):
    def __init__(self, model=None, criterion=None, use_res=True, inch=12, outch=1):
        
        super().__init__()
        ENCODER = 'resnet50'
        ENCODER_WEIGHTS = None
        ACTIVATION = None
        model = torch.load('/kaggle/input/bedrooms-last-10-4/model_centers_bed__100.pth', map_location=torch.device('cpu'))
        model.model.backbone.conv1 = nn.Conv2d(inch, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        model.model.classifier[4] = nn.Conv2d(256, outch, kernel_size=(1, 1), stride=(1, 1))
    
        self.model = model.cuda()
        
#         preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

        if criterion is None:
            criterion = nn.MSELoss()
        self.loss_func = criterion 
    
    def forward(self, x):
        x = self.model(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        sch = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": sch,
                "monitor": "train_loss",
            }
        }
    
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.loss_func(outputs, labels)
        self.log("train_loss", loss, on_epoch=True)
        return loss

    
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.loss_func(outputs, labels)
        self.log("val_loss", loss, on_epoch=True)

        # store an example output and label
        self.example_input = inputs[0]
        self.example_output = outputs[0]
        self.example_label = labels[0]

        return loss

    def on_train_epoch_end(self):
        print(f"Epoch: {self.current_epoch}")
        print(f"Train Loss: {self.trainer.callback_metrics['train_loss']}")
        print(f"Validation Loss: {self.trainer.callback_metrics['val_loss']}")
        
        plt.imshow(self.example_input.detach().cpu().permute((1, 2, 0))[:, :, [1, 4, 3]])
        plt.show()
        plt.imshow(self.example_label.detach().cpu().permute((1, 2, 0))[:, :, :])
        plt.show()
        plt.imshow(self.example_output.detach().cpu().permute((1, 2, 0))[:, :, 0])
        plt.show()



class DecoderCNN(nn.Module):
    def __init__(self):
       super().__init__()
        
       self.conv1 = nn.Conv2d(2560, 2048, kernel_size=3, padding=1)
       self.bn1 = nn.BatchNorm2d(2048)
    

       self.deconv1 = nn.ConvTranspose2d(2048, 512, kernel_size=4, stride=2, padding=1, output_padding=0)
       self.bn3 = nn.BatchNorm2d(512)

       self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, output_padding=0)  
       self.bn4 = nn.BatchNorm2d(256)
       
       self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, output_padding=0)
       self.bn5 = nn.BatchNorm2d(128)

       self.conv3 = nn.Conv2d(128, 1, kernel_size=3, padding=1)


    def forward(self, x):
       # Encoder 
       x = F.relu(self.bn1(self.conv1(x)))         
       x = F.relu(self.bn3(self.deconv1(x)))
       x = F.relu(self.bn4(self.deconv2(x)))  
       x = F.relu(self.bn5(self.deconv3(x)))
       x = torch.sigmoid(self.conv3(x))
        
       return x
        
class BedCentRegressor2(pl.LightningModule):
    def __init__(self, inch=12, state=None):
        super().__init__()
        
        self.state = state

        model = torch.load('/kaggle/input/bedrooms-last-10-4/model_centers_bed__100.pth', map_location=torch.device('cpu'))
        model.model.backbone.conv1 = nn.Conv2d(inch, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        cent_model_encoder = torch.nn.Sequential(*(list(model.model.children())[:-2]))
        
        self.encoder_main = cent_model_encoder
        self.encoder_mini = Encoder(inch=3)

        self.decoder_aio = DecoderCNN()
        self.decoder_bed = DecoderCNN()
        self.decoder_bath = DecoderCNN()
        self.decoder_kit = DecoderCNN()
        
        
        self.loss_func =  nn.MSELoss() 
        
    def set_state(self, state):
        self.state = state
    
    def forward_encoder_main(self, x):
        return self.encoder_main(x)["out"]
    
    def forward_encoder_mini(self, x):
        return self.encoder_mini(x)  
    
    def forward_decoder(self, x1, x2, state=None):
        x = torch.cat([x1, x2], dim=1)
        if state == None:
            x = self.decoder_aio(x)
        if state == 1:
            x = self.decoder_bed(x)        
        elif state == 2:
            x = self.decoder_bath(x)    
        elif state == 3:
            x = self.decoder_kit(x)
        return x
        
    def forward(self, x1, x2):
        x1 = self.encoder_main(x1)["out"]
        x2 = self.encoder_mini(x2)
        x = torch.cat([x1, x2], dim=1)
        
        if self.state == None:
            x = self.decoder_aio(x)
        if self.state == 1:
            x = self.decoder_bed(x)        
        elif self.state == 2:
            x = self.decoder_bath(x)    
        elif self.state == 3:
            x = self.decoder_kit(x)

        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        sch = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": sch,
                "monitor": "train_loss",
            }
        }
    
    def training_step(self, batch, batch_idx):
        inputs_1, inputs_2, labels, state = batch
        outputs = self(inputs_1, inputs_2)
        loss = self.loss_func(outputs, labels)
        self.log("train_loss", loss, on_epoch=True)
        return loss

    
    def validation_step(self, batch, batch_idx):
        inputs_1, inputs_2, labels, state = batch
        outputs = self(inputs_1, inputs_2)
        loss = self.loss_func(outputs, labels)
        self.log("val_loss", loss, on_epoch=True)

        # store an example output and label
        self.example_input_1 = inputs_1[0]
        self.example_input_2 = inputs_2[0]
        self.example_output = outputs[0]
        self.example_label = labels[0]

        return loss

    def on_train_epoch_end(self):
        print(f"Epoch: {self.current_epoch}")
        print(f"Train Loss: {self.trainer.callback_metrics['train_loss']}")
        print(f"Validation Loss: {self.trainer.callback_metrics['val_loss']}")
        
        plt.imshow(self.example_input_1.detach().cpu().permute((1, 2, 0))[:, :, [1, 2, 3]])
        plt.show()        
        plt.imshow(self.example_input_2.detach().cpu().permute((1, 2, 0))[:, :, [0, 1, 2]])
        plt.show()
        plt.imshow(self.example_label.detach().cpu().permute((1, 2, 0))[:, :, :])
        plt.show()
        plt.imshow(self.example_output.detach().cpu().permute((1, 2, 0))[:, :, 0])
        plt.show()
#         plt.imshow(self.example_output.detach().cpu().permute((1, 2, 0))[:, :, 1])
#         plt.show()
#         plt.imshow(self.example_output.detach().cpu().permute((1, 2, 0))[:, :, 2])
#         plt.show()
        

# class DecoderCNN(nn.Module):
#     def __init__(self):
#         super(DecoderCNN, self).__init__()
        
#         # Encoder layers
#         self.conv1 = nn.Conv2d(2560, 512, kernel_size=4, padding=1)
#         self.bn1 = nn.BatchNorm2d(512)
        
#         self.conv2 = nn.Conv2d(512, 256, kernel_size=4, padding=1)
#         self.bn2 = nn.BatchNorm2d(256)

#         # Decoder layers
#         self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=3, padding=2)
#         self.bn3 = nn.BatchNorm2d(128)

#         self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=3, padding=2)
#         self.bn4 = nn.BatchNorm2d(64)

#         self.conv3 = nn.Conv2d(64, 1, kernel_size=3, padding=1)

#     def forward(self, x):
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))

#         x = F.relu(self.bn3(self.deconv1(x)))
#         x = F.relu(self.bn4(self.deconv2(x)))

#         x = self.conv3(x)
#         return x

In [ ]:
DecoderCNN()(torch.zeros([1, 2560, 32, 32])).shape

In [ ]:
class RoomLayoutUNet(nn.Module):
    def __init__(self, n_channels=4, n_classes=11):
        super(RoomLayoutUNet, self).__init__()
        self.model = models.segmentation.deeplabv3_resnet101(pretrained=True, progress=True).cuda()
        print(self.model.backbone.conv1)
        self.model.backbone.conv1 = nn.Conv2d(n_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).cuda()
        self.model.classifier[4] = nn.Conv2d(256, n_classes, kernel_size=(1, 1), stride=(1, 1)).cuda()
        self.embedding = nn.Embedding(5, 1).cuda()

    def forward(self, x):
        output = self.model(x)['out']
        return output



In [ ]:
model = BedCentRegressor2().load_from_checkpoint("/kaggle/input/blacdoorwindow/xxyy_t/epoch=45-step=21114.ckpt")

In [ ]:
# model.decoder_bed = DecoderCNN()
# model.decoder_bath = DecoderCNN()
# model.decoder_kit = DecoderCNN()
# model.decoder_aio = DecoderCNN()

In [ ]:
# trainer.save_checkpoint("final.ckpt")

In [ ]:
for param in model.encoder_main.parameters():
    param.requires_grad = False
    
for param in model.encoder_mini.parameters():
    param.requires_grad = True 
    
for param in model.decoder_bed.parameters():
    param.requires_grad = True  
    
for param in model.decoder_kit.parameters():
    param.requires_grad = True   
    
for param in model.decoder_bath.parameters():
    param.requires_grad = True
    
model.decoder_bed = None
model.decoder_bath = None
model.decoder_kit = None
# model.decoder_bath.load_state_dict(model.decoder_bed.state_dict())
# model.decoder_kit.load_state_dict(model.decoder_bed.state_dict())

In [ ]:
model;

In [ ]:
import collections
from pytorch_lightning.loggers import Logger
from pytorch_lightning.utilities import rank_zero_only

class HistoryLogger(Logger):
    def __init__(self):
        super().__init__()
        self.history = collections.defaultdict(list)

    @property
    def name(self):
        return "HistoryLogger"

    @property
    def version(self):
        return "1.0"

    @rank_zero_only
    def log_hyperparams(self, params):
        # store hyperparameters in the history if you want
        self.history['hyperparams'] = vars(params)

    @rank_zero_only
    def log_metrics(self, metrics, step):
        for metric_name, metric_value in metrics.items():
            self.history[metric_name].append(metric_value)
            
logger = HistoryLogger()


In [ ]:
# cent_model = BedCentRegressor(inch=7, outch=3)
# # cent_model.model.model.backbone.conv1 = nn.Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# # cent_model.model.model.classifier[4] = nn.Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
# cent_model.load_state_dict(torch.load("/kaggle/input/blacdoorwindow/xxyy_2/epoch=11-step=8016.ckpt")["state_dict"])
# cent_model.cuda();

In [ ]:
# cent_model.model.model.backbone.conv1 = nn.Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# cent_model.model.model.classifier[4] = nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))

In [ ]:
# cent_model.model.model.backbone.conv1 = nn.Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [ ]:
import gc, torch

torch.cuda.empty_cache()
gc.collect()

In [ ]:
# trainer.save_checkpoint("aio_last.ckpt")

In [ ]:
# model.encoder_mini.conv1 = nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))

In [ ]:
def get_width(p):
    x1, y1, x2, y2 = p['inner'].bounds
    return max(x2-x1, y2-y1)

plans = [p for p in plans if get_width(p) <= 256]
plan_all = plans
train_plans, test_plans = train_test_split(plan_all, test_size=0.1, shuffle=True, random_state=1997)

batch = 32

v=1

train_plans = [i for i in train_plans if i['front_door'] and i['bedroom'] and i['bathroom'] and i['general'] ]
test_plans = [i for i in test_plans if i['front_door'] and i['bedroom'] and i['bathroom'] and  i['general'] ]

In [ ]:
state =  4


train_dl = DataLoader(PlanDataset(train_plans, state=state), batch_size=batch, shuffle=True,num_workers=2)
val_dl = DataLoader(PlanDataset(test_plans, state=state, val=v), batch_size=batch, shuffle=True, num_workers=2)

model.set_state(None)

checkpoint = ModelCheckpoint(dirpath='xxyy', monitor="val_loss", mode="min")

trainer = Trainer(max_epochs=1000, callbacks=[checkpoint], gpus=1, logger=logger, auto_lr_find=True)
trainer.fit(model, train_dl, val_dl)

In [ ]:
# cent_model = torchvision.models.segmentation.deeplabv3_resnet50(num_classes=1)
# cent_model.backbone.conv1 = nn.Conv2d(9, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# # cent_model.classifier[4] = nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
# cent_model = cent_model.cuda()

In [ ]:
cent_model = torch.load('/kaggle/input/bedrooms-last-10-4/model_centers_bed__100.pth')
cent_model.model.backbone.conv1 = nn.Conv2d(9, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
cent_model.model.classifier[4] = nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
cent_model = cent_model.cuda()

In [ ]:
loss_fn = nn.MSELoss()

from shapely.geometry import box
import geopandas as gpd
from tqdm.notebook import tqdm

lr=1e-4


def accuracy_2(y_true, y_pred):
    numerator = torch.sum(((y_pred>0.2) * y_true).bool().float())
    denominator = torch.sum(y_true)
    accuracy = numerator / denominator
    return accuracy

train_loss = []
val_loss = []
train_acc = []
val_acc = []

lr = lr * 0.98
optimizer = torch.optim.Adam(cent_model.parameters(), lr=lr)

for i in range(200_000):    
    for k, data in tqdm(enumerate(train_dl)):
        inputs, labels = data[0].cuda(), data[1].cuda()
        outputs = cent_model(inputs)
        loss = loss_fn(outputs[:, 0, :, :], labels[:, 0, :, :])   
        train_acc.append(accuracy_2(labels, outputs).detach().cpu().item())
        loss.backward()
        optimizer.step()
        train_loss.append(loss.detach().cpu().item())

        del loss
        del inputs
        del labels
        del outputs
        torch.cuda.empty_cache()
        gc.collect()

        if k % 100 == 0:
            lr = lr * 0.8
            if lr <= 1e-5:
                lr = 1e-5
            optimizer = torch.optim.Adam(cent_model.parameters(), lr=lr)
            with torch.no_grad():
                for data in val_dl:
                    inputs, labels = data[0].cuda(), data[1].cuda()
                    outputs = cent_model(inputs)
                    loss = loss_fn(outputs[:, 0, :, :], labels[:, 0, :, :])
                    val_acc.append(accuracy_2(labels, outputs).detach().cpu().item())
                    val_loss.append(loss.detach().cpu().item())

                plt.imshow(inputs[0].detach().cpu().permute((1, 2, 0))[:, :, [1, 3, 4]])
                plt.show()

                plt.imshow(outputs[0].detach().cpu()[0])
                plt.show()

                plt.imshow(labels[0].detach().cpu()[0])
                plt.show()

                print(np.mean(train_loss))
                print(np.mean(val_loss))            
                print(np.mean(train_acc))
                print(np.mean(val_acc))

                train_loss = []
                val_loss = []
                train_acc = []
                val_acc = []
                
    optimizer = torch.optim.Adam(cent_model.parameters(), lr=lr)
#     torch.save(cent_model.cuda(), f'/kaggle/working/bedroom_.pth')

In [ ]:
import gc, torch

del loss
del inputs
del labels
del outputs

torch.cuda.empty_cache()
gc.collect()

In [ ]:
del outputs
torch.cuda.empty_cache()
gc.collect()

In [ ]:
del loss
del inputs
del outputs

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [ ]:
download_file("/kaggle/working/xxyy", "xxyy_70ep.zip")

In [ ]:
ch

In [ ]:
torch.save(xxyy_model, "70ep_xxyy_model.pth")

In [ ]:

plt.plot(trainer.logger.history['train_loss_epoch'][1:], label='train_loss')
plt.plot(trainer.logger.history['val_loss'][1:], label='val_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
checkpoint.best_model_path

In [ ]:
best_model = XXYYRegressor().load_from_checkpoint(checkpoint.best_model_path)

In [ ]:
torch.set_grad_enabled(True)


In [ ]:
torch.save(best_model, "70ep_xxyy_model.pth")

In [ ]:
from IPython.display import FileLink


for epoch in range(num_epochs):
    print(f'_________________________')

    
    model.eval() 
    running_loss = 0.0 
    with torch.no_grad():  
        for i, (inputs, labels) in tqdm(enumerate(test_dl)): 
            inputs, labels = inputs.cuda(), labels.cuda().float() / 60.0
            outputs = model(inputs)  
            loss = criterion(outputs, labels)  
            running_loss += loss.item() * inputs.size(0) 
    epoch_loss = running_loss / len(test_dl.dataset)
    print(outputs[0]*100, labels[0]*100)
    print(f'Validation Loss: {epoch_loss:.4f}')
    
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'best_model.pth')
        # torch.save(model.cuda(), f'/kaggle/working/model_bed_bath.pth')
    display(FileLink("best_model.pth"))

        
    model.train()  # set the model to training mode
    running_loss = 0.0
    for i, (inputs, labels) in tqdm(enumerate(train_dl), total=len(train_dl)):  # iterate over the training data
        try:
            inputs, labels = inputs.cuda(), labels.cuda().float() / 60.0
            optimizer.zero_grad() 
            outputs = model(inputs) 
            loss = criterion(outputs, labels)
            loss.backward()  
            optimizer.step()  
            running_loss += loss.item() * inputs.size(0) 
        except:
            ...
    epoch_loss = running_loss / len(train_dl.dataset)  
    print(f'Train Loss: {epoch_loss:.4f}')
    
#     lr /= 3
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    



In [ ]:
model_2

In [ ]:
import copy

class CustomUnet(SegmentationModel):
    def __init__(self, encoder_name="", mini_encoder=None, encoder=None, decoder=None, concat_size=5, big_image_channels=0, decoders_num=6,**kwargs):
        super().__init__()
        if encoder is not None:
            self.encoder = encoder
        else:
            self.encoder = get_encoder(encoder_name, **kwargs)
            
            
        decoder_channels = (256, 128, 64, 32, 16)
        
        enc_channels = list(self.encoder.out_channels)
        
        enc_channels[-1] += concat_size + (64 if big_image_channels else 0)
        
        self.decoders = [UnetDecoder(
                encoder_channels=enc_channels,
                decoder_channels=decoder_channels,
                n_blocks=5,
                use_batchnorm=True,
                center=True,
                attention_type=None
            ).cuda() for nn in range(decoders_num)]
        
        if decoder is not None:
            for nn in range(decoders_num):
                x = self.decoders[nn].load_state_dict(copy.deepcopy(decoder.state_dict()))
                print(x)
        self.segmentation_head = SegmentationHead(
            in_channels=decoder_channels[-1],
            out_channels=kwargs.get("classes", 1),
            activation=None,
            kernel_size=3,
        )
        if big_image_channels and mini_encoder is None:
            self.mask_encoder = MiniEncoder(big_image_channels)
        elif mini_encoder is not None:
            self.mask_encoder = mini_encoder
        
        else:
            self.mask_encoder = None


    def forward(self, x, float_tensor, bedn, other=None, decoder_n=0):
        features = self.encoder(x)
        float_tensor = float_tensor.view(-1, 1, 1, 1).expand_as(features[-1])[:, 0, :, :].unsqueeze(1)
        onehot = torch.zeros(float_tensor.shape[0], 4, float_tensor.shape[2], float_tensor.shape[3]).cuda()
        if self.mask_encoder is not None: 
            other_inputs = self.mask_encoder(other)
            bottleneck = torch.cat((features[-1], float_tensor, bedn, other_inputs), 1)
        else:
            bottleneck = torch.cat((features[-1], float_tensor, bedn), 1)
            
        features[-1] = bottleneck
        x = self.decoders[decoder_n](*features)
        x = self.segmentation_head(x)
        return x


In [ ]:
# model_2 = CustomUnetPlusPlus().cuda()

In [ ]:

# model_2 = torch.load("/kaggle/input/unet-custom-first-bedroom/bedroom_second_unet_22.pth").cuda()

# for name, para in model_2.encoder.named_parameters():
#      para.requires_grad = False
        
# model.encoder = model_2.encoder
# orig_decoder = model_2.decoder
# orig_decoder.center = model.decoder.center
# orig_decoder.blocks[0] = model.decoder.blocks[0]
# model.decoder = orig_decoder


# for name, para in model.decoder.named_parameters():
#      para.requires_grad = True

In [ ]:
# model.mask_encoder = MaskBranch(2).cuda()


In [ ]:



# model_2 = torch.load('/kaggle/input/unet-custom-first-bedroom/bedroom_second_unet_1000.pth').cuda()
# 

In [ ]:
# outputs = model(inputs, areas, bedn, bed1)


In [ ]:
0

In [ ]:
# model =  torch.load("/kaggle/input/bedroom-second-stage-5-2023-v1/bed_second_stage_model.pth", map_location=torch.device('cpu')).eval();
# # model_aug.resnet.segmentation_head[0] = nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
# model.resnet.encoder.conv1 = nn.Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# model.unet.encoder.conv1 = nn.Conv2d(13, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# model = model.cuda()

In [ ]:
# !wget https://www.kaggleusercontent.com/kf/134768889/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..0PV9B-hHYb7wLzPN4Gkbpg.6vTv5WMXPs5joUBU5lY7VjgPXsuAqcA-4UQJrnrJWdMPb5gRr6_-bjz4gbGM1xJuXRgVOVbVP_1856_yNVk959xUSrRiSeiT6EfpVE2qMTqnSt5JUGx4g6XzTJGvyOpg77kIyUpSjQBJnQQbhaDBMS8AM1tLh5WiHhj-K2beXiUA3kK8J--FiuXGV37RFG8a2EXF12ip8ZDQSPHF-0v1zKwOJFg-tJI3-yG07d-j-McsH3gQ83FXYAIS8WtmhKpyiYVo2gvk9t8jn_M00dN9GOCX1vfCHVRE0M8lYhmHGS2gAjKwXJRd55D2w54gFEGz1m1UATunONOGrJ_KVTtVrswF5xrSdxdC2JKanVpjXQ7PMr1ms3J9S_05j_2mPk8aM9Mo8ikAs9XdX4ySu-PouZGBNK2AllIAw6e5FRnAJCXdKpHGcmNTNJRddz6QzljX4q5Z875y-ukb0zMmnUdpaYnIg10_KBe1N7pkiWlpaLDTmqROa6Il5UpdbMaf0x-5i8yQvo31kbg1dbS1E-zHm_V17Iyg9JxvZgBe88jdIeRozlBgcz1UWqPWMSus5xoF4uIM1BO6dau4edNTytq7PlUTFi8yJ-sChym6YVoXA0Lx_Qu-oKaeo1rPwKvg8TzjgGHFzdaDvAk2CjMb1uFuYHymTxpNnPkbNENa8fhjRa8.af3ApNarl3B5436LI-zGDw/bedroom_second_unet_16.pth

In [ ]:
import torch
# model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
#     in_channels=10, out_channels=1, init_features=32, pretrained=False).cuda()

# model = torch.load('/kaggle/input/second-room-rplan/bedroom_second_unet_16.pth').cuda()


In [ ]:
# model.encoder1.enc1conv1 = nn.Conv2d(11, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False).cuda()

In [ ]:
# model.encoder1.enc1conv1 = nn.Conv2d(10, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False).cuda()
# model.conv = nn.Conv2d(32, 1, kernel_size=(1, 1), stride=(1, 1)).cuda()

In [ ]:
def get_width(plan):
    w = plan['inner'].bounds[2] - plan['inner'].bounds[0]
    return w

plans = [p for p in plans if get_width(p) <= 256]

train_plans_o, test_plans_o = train_test_split(plans, test_size=0.02, shuffle=True, random_state=1997)
train_plans_r, test_plans_r = train_test_split(rplans, test_size=0.02, shuffle=True, random_state=1997)

batch = 12
v=0
train_plans = [i for i in train_plans_o if i['front_door'] and i['bedroom'] and len(i['bedroom'].geoms)>= 1  and i['bathroom'] and i['general']]
test_plans = [i for i in test_plans_o if i['front_door'] and i['bedroom'] and len(i['bedroom'].geoms) >= 1  and i['bathroom'] and  i['general']]
bed1_traindl = DataLoader(PlanDataset(train_plans, task='bed1'), batch_size=batch, shuffle=True,num_workers=2)
bed1_valdl = DataLoader(PlanDataset(test_plans, task='bed1', val=v), batch_size=batch, shuffle=True, num_workers=2)

train_plans = [i for i in train_plans_o if i['front_door'] and i['bedroom'] and len(i['bedroom'].geoms)>= 2 and i['bathroom'] and i['general'] ]
test_plans = [i for i in test_plans_o if i['front_door'] and i['bedroom'] and len(i['bedroom'].geoms)>= 2  and i['bathroom'] and  i['general']]
bed2_traindl = DataLoader(PlanDataset(train_plans, task='bed2'), batch_size=batch, shuffle=True,num_workers=2)
bed2_valdl = DataLoader(PlanDataset(test_plans, task='bed2', val=v), batch_size=batch, shuffle=True, num_workers=2)

train_plans = [i for i in train_plans_o if i['front_door'] and i['bedroom'] and len(i['bedroom'].geoms) >= 3  and i['bathroom'] and i['general']]
test_plans = [i for i in test_plans_o if i['front_door'] and i['bedroom'] and len(i['bedroom'].geoms) >= 3  and i['bathroom'] and  i['general']]
bed3_traindl = DataLoader(PlanDataset(train_plans, task='bed3'), batch_size=batch, shuffle=True,num_workers=2)
bed3_valdl = DataLoader(PlanDataset(test_plans, task='bed3', val=v), batch_size=batch, shuffle=True, num_workers=2)

train_plans = [i for i in train_plans_o if i['front_door'] and i['bedroom'] and len(i['bathroom'].geoms) >= 1  and i['bathroom'] and i['general'] ]
test_plans = [i for i in test_plans_o if i['front_door'] and i['bedroom'] and len(i['bathroom'].geoms) >= 1  and i['bathroom'] and  i['general']]
bath1_traindl = DataLoader(PlanDataset(train_plans, task='bath1'), batch_size=batch, shuffle=True,num_workers=2)
bath1_valdl = DataLoader(PlanDataset(test_plans, task='bath1', val=v), batch_size=batch, shuffle=True, num_workers=2)

train_plans = [i for i in train_plans_o if i['front_door'] and i['bedroom'] and len(i['bathroom'].geoms) >= 2   and i['bathroom'] and i['general'] ]
test_plans = [i for i in test_plans_o if i['front_door'] and i['bedroom'] and len(i['bathroom'].geoms) >= 2  and i['bathroom'] and  i['general']]
bath2_traindl = DataLoader(PlanDataset(train_plans, task='bath2'), batch_size=batch, shuffle=True,num_workers=2)
bath2_valdl = DataLoader(PlanDataset(test_plans, task='bath2', val=v), batch_size=batch, shuffle=True, num_workers=2)

train_plans = [i for i in train_plans_o if i['front_door'] and i['bedroom'] and len(i['bathroom'].geoms) >= 3   and i['bathroom'] and i['general']]
test_plans = [i for i in test_plans_o if i['front_door'] and i['bedroom'] and len(i['bathroom'].geoms) >= 3  and i['bathroom'] and  i['general']]
bath3_traindl = DataLoader(PlanDataset(train_plans, task='bath3'), batch_size=batch, shuffle=True,num_workers=2)
bath3_valdl = DataLoader(PlanDataset(test_plans, task='bath3', val=v), batch_size=batch, shuffle=True, num_workers=2)

train_plans = [i for i in train_plans_r if i['front_door'] and i['bedroom'] and  i['bathroom'] and i['kitchen']]
test_plans = [i for i in test_plans_r if i['front_door'] and i['bedroom'] and i['bathroom'] and  i['kitchen']]
kit_traindl = DataLoader(PlanDataset(train_plans[:47000], task='kit'), batch_size=batch, shuffle=True,num_workers=2)
kit_valdl = DataLoader(PlanDataset(test_plans[:100], task='kit', val=v), batch_size=batch, shuffle=True, num_workers=2)

len(train_plans)

In [ ]:
train_plan_datasets = [bed1_traindl, bed2_traindl,bed3_traindl,bath1_traindl,bath2_traindl,bath3_traindl, kit_traindl]
val_plan_datasets = [bed1_valdl, bed2_valdl,bed3_valdl,bath1_valdl,bath2_valdl,bath3_valdl, kit_valdl]

[len(i) for i in train_plan_datasets] , [len(i) for i in val_plan_datasets] 

In [ ]:
max([len(i) for i in train_plan_datasets]), max([len(i) for i in val_plan_datasets])

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

def accuracy_2(y_true, y_pred):
    numerator = torch.sum(((y_pred>0.2) * y_true).bool().float())
    denominator = torch.sum(y_true)
    accuracy = numerator / denominator
    return accuracy > 0.5

In [ ]:
gc.collect()
torch.cuda.empty_cache()

i = random.randint(0, 31)
x = next(iter(test_data))

plt.imshow( model(x[0])[i] )
plt.show()
plt.imshow(x[0][i][:, :, 8:11] + 2* x[1][i][:, :, :])
plt.show()

In [ ]:
ll = 1e-5

In [ ]:
ll = ll * 0.97
ll

In [ ]:
# for name, para in model.encoder.named_parameters():
#      para.requires_grad = False

In [ ]:
max([len(i) for  i in val_plan_datasets])

In [ ]:

import uuid

from matplotlib import pyplot as plt

import torch
from typing import Iterable

import cv2
import numpy as np
from pandas import Series
from shapely.geometry import MultiPolygon, Polygon, Point, box
import matplotlib.pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray
from skimage import io


def imfy(img):
    return (np.clip(img, 0, 1) * 255).astype(np.uint8)


def get_rects(imgr, centroids):
    bounds = get_mask(centroids, point_s=8).astype(np.uint8)
    img = (imgr).astype(np.uint8)
    #     img[bounds>0] = 0

    img[np.where(bounds == 0)] = 0
    # plt.imshow(img)
    # plt.show()

    shapes = []
    cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for i, c in enumerate(cnts[0]):
        M = cv2.moments(c)
        box = cv2.boundingRect(c);
        x1, y1, w, h = box
        x2, y2 = x1 + w, y1 + h

        ii = img[y1:y2, x1:x2]

        intensity = np.sum(ii > np.median(ii))

        if not M["m00"]:
            continue
        cx = M["m10"] / M["m00"]
        cy = M["m01"] / M["m00"]
        shapes.append([cx, cy, intensity])

    return [Point(*i[:2]) for i in sorted(shapes, key=lambda x: x[2], reverse=True)]


def get_rects_2(channel, gcenters=True):
    a = channel.copy()

    a[a < 0.9] = 0
    a[a > 1] = 1

    a = (255 * a).astype(np.uint8)

    _, thresh = cv2.threshold(a, 0, 255, cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    final_c = []
    centers = []
    for c in contours:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.01 * perimeter, True)
        x, y, w, h = cv2.boundingRect(approx)
        shapely_box = box(x, y, x + w, y + h)
        final_c.append(shapely_box.buffer(5, join_style=2, cap_style=2))

        mask = np.zeros((256, 256))
        mask = cv2.drawContours(mask, [c], -1, 255, -1)

        rms = channel.copy()
        rms[mask == 0] = 0

        M = cv2.moments(rms)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centers.append(Point(cX, cY))

    if gcenters:
        return centers
    return MultiPolygon(final_c).buffer(0)


# def accuracy_2(y_true, y_pred):
#     acc = []
#     for i in range(len(y_true)):
#         yt = y_true[i]
#         yp = y_pred[i]
#         yp = yp[0].detach().numpy()
#         yt = yt[0].detach().numpy()
#         im = imfy(yp)
#         img = cv2.GaussianBlur(im, (15, 15), 5)
#         blobs_log = blob_log(img, min_sigma=6, max_sigma=30, threshold=0.01)
#         bed_centroids = [Point(i[1], i[0]) for i in blobs_log]
#         try:
#             bed_centroids = get_rects(im, bed_centroids)[0]
#         except:
#             bed_centroids = []
#         bed_centroids_in = get_mask(bed_centroids, point_s=8)
#         numerator = np.sum(((bed_centroids_in>0.2) * yt) > 0 )
#         denominator = np.sum(yt)
#         accuracy = numerator / denominator
        
#         acc.append(accuracy > 0.5)
#     return np.mean(acc)


def accuracy_2(y_true, y_pred):
    numerator = torch.sum(((y_pred>0.2) * y_true).bool().float())
    denominator = torch.sum(y_true)
    accuracy = numerator / denominator
    return accuracy

In [ ]:
# for name, para in model_2.encoder.named_parameters():
#     para.requires_grad = True
    
# for name, para in model_2.mask_encoder.named_parameters():
#     para.requires_grad = True 
    
# for dec in model_2.decoders:
#     for name, para in dec.named_parameters():
#         para.requires_grad = True 
    


In [ ]:
try:
    print(model_2.steps)
except:
    model_2.steps=0

In [ ]:
model_2.steps

In [ ]:
import gc


# del inputs
# del labels 
# del i
# del d
# del outputs
# del train_loss 
# del val_loss 
# del train_acc 
# del val_acc
# del loss


gc.collect()
torch.cuda.empty_cache()


In [ ]:
loss_fn = nn.MSELoss()

from shapely.geometry import box
import geopandas as gpd
import pygeos
import tqdm
from pygeos.creation import box

lr=1e-4

os.makedirs('sample_output', exist_ok=True)
model_2 = model_2.train()

train_loss = []
val_loss = []
train_acc = []
val_acc = []

t_itter = {i: [iter(val), val] for i, val in enumerate(train_plan_datasets)}
lr = lr * 0.98
optimizer = torch.optim.Adam(model_2.parameters(), lr=lr)

# for i in tqdm.tqdm(range(200000)):
for i in range(200_000):
    
    epoch = i // 2000  
    
    for idx, (dit, dg) in t_itter.items():
        optimizer.zero_grad()

        try:
            d = next(dit)
        except:
            dit = iter(dg)
            t_itter[idx] = (dit, dg)
            d = next(dit)
        inputs, labels = d[0].cuda(), d[1].cuda()
        outputs = model_2(inputs)
        model_2.steps += 1
        loss = loss_fn(outputs, labels)   

        train_acc.append(accuracy_2(labels, outputs).detach().cpu().item())

        loss.backward()
        train_loss.append(loss.detach().cpu().item())
        optimizer.step()


    del inputs
    del labels 
    del d
    del loss
    del outputs
    gc.collect()
    torch.cuda.empty_cache()
    optimizer.step()


    if i % 300 == 0:
        lr = lr * 0.8
        if lr <= 1e-5:
            lr = 1e-5
        optimizer = torch.optim.Adam(model_2.parameters(), lr=lr)
        with torch.no_grad():
            v_itter = {i: [iter(val), val] for i, val in enumerate(val_plan_datasets)}
            for j in range(28):
                for idx, (dit, dg) in v_itter.items():
                    try:
                        d = next(dit)
                    except:
                        dit = iter(dg)
                        v_itter[idx] = (dit, dg)
                        d = next(dit)
                    inputs, labels = d[0].cuda(), d[1].cuda()

                    outputs = model_2(inputs)
                    loss = loss_fn(outputs, labels) 
                    val_loss.append(loss.detach().cpu().item())
                    val_acc.append(accuracy_2(labels, outputs).detach().cpu().item())

                    if j == 0:
                        inp1 = inputs[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,[19, 1, 18]]
                        inp1[:, :, 1] = inp1[:, :, 1] - inp1[:, :, 0]
                        inp1[:, :, 1] = inp1[:, :, 1] - inp1[:, :, 2]
                        inp1[:, :, [0, 2]] += inputs[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,[0, 0]]
                        inp2 = labels[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,:]
                        inp3 = outputs[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,:]

#                             plt.imshow( inp1, origin='lower' )
#                             plt.show()
#                             plt.imshow( inp2, origin='lower' )
#                             plt.show()
#                             plt.imshow( inp3, origin='lower' )
#                             plt.show()
# create a figure and its subplots
                        fig, axs = plt.subplots(1, 4, figsize=(20, 5))

                        # display each image on each subplot
                        axs[0].imshow(inp1, cmap='inferno' )
                        axs[1].imshow(inp2, cmap='inferno' )
                        inp3[inp1[:, :, 0]>0] = 0
                        inp3[inp1[:, :, 2]>0] = 0
                        axs[2].imshow(inp3, cmap='inferno' )

                        im = imfy(inp3)
                        img = cv2.GaussianBlur(im, (15, 15), 5)
                        blobs_log = blob_log(img, min_sigma=4, max_sigma=30, threshold=0.01)
                        bed_centroids = [Point(i[1], i[0]) for i in blobs_log]
                        try:
                            bed_centroids = get_rects(im, bed_centroids, door)[0]
                        except:
                            bed_centroids = []
                        bed_centroids_in = get_mask(bed_centroids, point_s=5)
#                         plt.imshow( bed_centroids_in/, origin='lower')
                        axs[3].imshow(bed_centroids_in, cmap='inferno' )

                        plt.show()

        print('Epoch {} - Train Loss: {:.6f} | Val Loss: {:.6f} | Train accuracy: {:.6f} | Val accuracy {:.6f} '.format(epoch+1, np.mean(train_loss), np.mean(val_loss), np.mean(train_acc), np.mean(val_acc)))
        val_loss = []
        train_loss = []
        train_acc = []
        val_acc = []


#     if i % 100 == 0:
#         torch.save(model_2.cuda(), f'/kaggle/working/bedroom_second_unet_{epoch}.pth')

    if i%100 == 0:
        optimizer = torch.optim.Adam(model_2.parameters(), lr=lr)
        torch.save(model_2.cuda(), f'/kaggle/working/bedroom_{epoch}.pth')

In [ ]:
aa = inp3>0

In [ ]:
# model_2 = model_2.eval()

## inputs

In [ ]:
# model_2.train();

In [ ]:
def get_input(inner, door, bedn, bathn, task, area, bedrooms=[], bathrooms=[]):

    s = 0.8

    perturb_range = 0


    inner = perturb_polygon(inner , x_range=(-perturb_range, perturb_range), y_range=(-perturb_range, perturb_range)).buffer(0)

    noif = 2

        
    no_bedrooms = bedn
    no_bathrooms = bathn


    x = np.zeros((256, 256, 20))

    front = door.centroid
    

    bedroomsc = bedrooms
    bathroomsc = bathrooms
    
    x[:,:,0] = get_mask(front, (256, 256), point_s=7) > 0
    x[:,:,1] = get_mask(inner, (256, 256), point_s=10) > 0
    
    if 'kit' not in task:
        x[:, :, nd[task] + 2] = 1.0 # 2, 3, 4, 5 ,6, 7

    bed_start = 9 # 9 10 11
    bath_start = 12 # 12 13 14
    num_start = 15 # 15 16 17
    bed_m, bath_m = 18, 19
    
    if 'bed' in task:
        bnn = int(task[3])
        x[:,:,bnn + bed_start - 1] = 1.0
        x[:,:,num_start + no_bedrooms - 1][:bnn] = 1.0
        x[:,:,bed_m] =  get_mask(bedroomsc[:bnn-1], (256, 256), point_s=6) > 0

    elif 'bath' in task:
        btn = int(task[4])
        x[:,:,btn + bath_start - 1] = 1.0
        x[:,:,num_start + no_bathrooms - 1] = 1.0
        x[:,:,bath_m] =  get_mask(bathroomsc[:btn-1], (256, 256), point_s=6) > 0
        x[:,:,bed_m] =  get_mask(bedroomsc, (256, 256), point_s=6) > 0

    elif 'kit' in task:
        x[:,:,bath_m] =  get_mask(bathroomsc, (256, 256), point_s=6) > 0
        x[:,:,bed_m] =  get_mask(bedroomsc, (256, 256), point_s=6) > 0

#     area += random.randint(-10, 10)

    x[:,:,8] = norm_area(area)


    return torch.from_numpy(x).permute(2, 0, 1).float()




In [ ]:
def get_input_wh(current, inner, door, bedrooms=[], bathrooms=[], kitchen=[]):

    s = 0.8


    x = np.zeros((256, 256, 4))

    front = door.centroid
    

    bedroomsc = bedrooms
    bathroomsc = bathrooms
    
    x[:,:,0] = get_mask(front, (256, 256), point_s=7) > 0
    x[:,:,1] = get_mask(inner, (256, 256), point_s=10) > 0
    
    data =  bedrooms + bathrooms + kitchen


    x[:,:,2] =  get_mask(data, (256, 256), point_s=6) > 0
    x[:,:,3] =  get_mask(current, (256, 256), point_s=6) > 0


    return torch.from_numpy(x).permute(2, 0, 1).float()




In [ ]:
import re
from shapely.geometry import Point
from shapely import from_wkt

def parse_points(s):
    numbers = list(map(int, re.findall(r'\d+', s)))
    points = [Point(numbers[i:i+2]) for i in range(0, len(numbers), 2)]
    
    return points



a = """
[<POINT (59 156)>, <POINT (70 90)>]
[<POINT (103 166)>, <POINT (131 75)>]
[<POINT (132 165)>]
POLYGON ((230.4 84.78348415828609, 230.4 78.55408415068374, 205.48239996959055 78.55408415068374, 205.48239996959055 81.66878415448491, 230.2295366510251 81.66878415448491, 230.2295366510251 84.78348415828609, 230.4 84.78348415828609))
POLYGON ((32.03489108965486 64.93806732138222, 155.58480001102853 64.93806732138222, 155.58480001102853 81.66878415448491, 230.2295366510251 81.66878415448491, 230.2295366510251 191.06193267861778, 25.599999999999994 191.06193267861778, 25.599999999999994 112.5562613848283, 32.03489108965486 112.5562613848283, 32.03489108965486 64.93806732138222))

""".split('\n')

d = [i for i in a if 'POL' in i]
door, inner = min(d, key=lambda x: len(x)), max(d, key=lambda x: len(x))
door = from_wkt(door)
inner = from_wkt(inner)

p = [i for i in a if 'POI' in i]
bed, bath, kit = [parse_points(i) for i in p]

alless = bed + kit + bath


In [ ]:
boxes = []

model.eval()
for i in alless:
    cx, cy = i.coords[0]
    x = get_input_wh(i, inner, door, bed , bath , kit).unsqueeze(0)
    
    y = model(x).detach().cpu().numpy()[0]
    
    w, h = y*60
    
    x1, x2, y1, y2 = cx - w/2, cx +w/2, cy-h/2, cy+h/2 
    b = box(x1, y1, x2, y2)
    boxes.append(b)

boxes = [i.intersection(inner.buffer(-3, join_style=2, cap_style=2)) for i in boxes]

boxes = sorted(boxes, key=lambda x: x.area, reverse=True)

for i in range(len(boxes)-1):
    new_b = boxes[i]
    for j in range(i + 1, len(boxes)):
        new_b = new_b.difference(boxes[j].buffer(3, join_style=2, cap_style=2))
    boxes[i] = new_b.buffer(-5, join_style=2, cap_style=2).buffer(5, join_style=2, cap_style=2)
    boxes[j] = boxes[j].buffer(-5, join_style=2, cap_style=2).buffer(5, join_style=2, cap_style=2)


gpd.GeoSeries([inner] + boxes).plot(cmap="Dark2_r")

In [ ]:
boxes = []

xxyy_model.eval()
for i in alless:
    cx, cy = i.coords[0]
    x = get_input_wh(i, inner, door, bed , bath , kit).cuda().unsqueeze(0)
    
    y = xxyy_model(x).detach().cpu().numpy()[0]
    
#     w, h = y*60
    
#     x1, x2, y1, y2 = cx - w/2, cx +w/2, cy-h/2, cy+h/2 
    x1, y1, x2, y2 = y * 256
    b = box(x1, y1, x2, y2)
    boxes.append(b)

# boxes = [i.intersection(inner.buffer(-3, join_style=2, cap_style=2)) for i in boxes]

# boxes = sorted(boxes, key=lambda x: x.area, reverse=True)

# for i in range(len(boxes)-1):
#     new_b = boxes[i]
#     for j in range(i + 1, len(boxes)):
#         new_b = new_b.difference(boxes[j].buffer(3, join_style=2, cap_style=2))
#     boxes[i] = new_b.buffer(-5, join_style=2, cap_style=2).buffer(5, join_style=2, cap_style=2)
#     boxes[j] = boxes[j].buffer(-5, join_style=2, cap_style=2).buffer(5, join_style=2, cap_style=2)


gpd.GeoSeries([inner] + boxes).plot(cmap="Dark2_r")

In [ ]:
import numpy as np
from scipy.signal import convolve2d


def get_rects(imgr, centroids, door, get_max=True):
    bounds = get_mask(centroids, point_s=8).astype(np.uint8)
    d = ~get_mask(door.centroid, point_s=30).astype(np.uint8)
#     imgr[:, :, 0] = (imgr[:, :, 0] - d).astype(np.uint8)
#     plt.imshow(imgr)
#     plt.gca().invert_yaxis() 
#     plt.show()
    img = (imgr).astype(np.uint8)
    img[np.where(bounds == 0)] = 0

    if get_max:
        kernel = np.ones((5, 5))
        convolved = convolve2d(imgr.reshape((256, 256)), kernel, mode='valid')
        y, x = np.unravel_index(np.argmax(convolved), convolved.shape)
        return [Point(x, y)]

    shapes = []
    cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for i, c in enumerate(cnts[0]):
        M = cv2.moments(c)
        box = cv2.boundingRect(c);
        x1, y1, w, h = box
        x2, y2 = x1 + w, y1 + h

        ii = img[y1:y2, x1:x2]
        
        ii = ii * (ii >= np.median(imgr))
        intensity = np.sum(ii)

        if not M["m00"]:
            continue
        cx = M["m10"] / M["m00"]
        cy = M["m01"] / M["m00"]
        shapes.append([cx, cy, intensity])

    return [Point(*i[:2]) for i in sorted(shapes, key=lambda x: x[2], reverse=True)]


In [ ]:
i1 = None
def get_centroid(inner, door, bedn, bathn, task, area, bedrooms=[], bathrooms=[], plot=True):
    global i1
    x1 = get_input(inner=inner, door=door, bedn=bedn, bathn=bathn, task=task, area=area,
                   bedrooms=bedrooms, bathrooms=bathrooms).unsqueeze(0).cuda()
    o1 = model_2(x1) * x1[:, 1, :, :]

    inp1 = x1[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,[19, 1, 18]]
    inp1[:, :, 1] = inp1[:, :, 1] - inp1[:, :, 0]
    inp1[:, :, 1] = inp1[:, :, 1] - inp1[:, :, 2]
    inp1[:, :, [0, 2]] += x1[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,[0, 0]]
    inp3 = o1[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,:]
    
    if task=='bed1':
        i1 = inp3
    im =inp3
    img = cv2.GaussianBlur(im, (15, 15), 5)
    blobs_log = blob_log(img, min_sigma=4, max_sigma=30, threshold=0.01)
    bed_centroids = [Point(i[1], i[0]) for i in blobs_log]

    bed_centroids = get_rects(im, bed_centroids, door)[0]

            
    if plot:
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # display each image on each subplot
        axs[0].imshow(inp1, cmap='inferno' )
        axs[0].set_title("Input")
        
        axs[1].imshow(inp3, cmap='inferno' )
        axs[1].set_title("Output")

        bed_centroids_in = get_mask(bed_centroids, point_s=5)
        axs[2].imshow(bed_centroids_in, cmap='inferno' )
        axs[2].set_title("Segmented output")

        for a in axs:
            a.invert_yaxis()
            
        fig.suptitle(task, fontsize=16)  # set title for the whole figure
        plt.show()
        
    return bed_centroids

bedrooms_predicted_centroids = []
bathrooms_predicted_centroids = []
kitchen_predicted_centroids = []
bed_n = 2
bath_n = 2
area=140
for i in range(bed_n):
    task = f"bed{i+1}"
    c = get_centroid(inner, door, bedn=bed_n, bathn=bath_n, task=task, area=area, bedrooms=bedrooms_predicted_centroids, bathrooms=bathrooms_predicted_centroids)
    bedrooms_predicted_centroids.append(c)

for i in range(bath_n):
    task = f"bath{i+1}"
    c = get_centroid(inner, door, bedn=bed_n, bathn=bath_n, task=task, area=area, bedrooms=bedrooms_predicted_centroids, bathrooms=bathrooms_predicted_centroids)
    bathrooms_predicted_centroids.append(c)
    
task = f"kit"
c = get_centroid(inner, door, bedn=bed_n, bathn=bath_n, task=task, area=area, bedrooms=bedrooms_predicted_centroids, bathrooms=bathrooms_predicted_centroids)
kitchen_predicted_centroids.append(c)    
# task = f"kit"
# c = get_centroid(inner, door, bedn=bed_n, bathn=bath_n, task=task, area=area, bedrooms=bedrooms_predicted_centroids, bathrooms=kitchen_predicted_centroids + bathrooms_predicted_centroids)
# kitchen_predicted_centroids.append(c)

In [ ]:
colors = ['Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'winter', 'winter_r']


In [ ]:


gpd.GeoSeries([inner, door] + [GeometryCollection(i) for i in [bedrooms_predicted_centroids, bathrooms_predicted_centroids, kitchen_predicted_centroids] ]).plot(cmap="plasma", markersize=100 )

plt.show()


print(bedrooms_predicted_centroids)
print(bathrooms_predicted_centroids)
print(kitchen_predicted_centroids)
print(door)
print(inner)

In [ ]:
with torch.no_grad():
    for j, (inputs, labels) in  enumerate(plan_dataloader_val):
        inputs, labels = inputs.cuda(),  labels.cuda()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        val_loss.append(loss.detach().cpu().item())
        val_acc.append(np.mean(accuracy_2(labels.cpu(), outputs.cpu())))

        if j == 0:
            inp1 = inputs[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,[0, 1, 1]]
            inp2 = labels[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,:]
            inp3 = outputs[0].permute(1, 2, 0).cpu().detach().numpy()[:,:,:]
        break


print('Epoch {} - Train Loss: {:.6f} | Val Loss: {:.6f} | Train accuracy: {:.6f} | Val accuracy {:.6f} '.format(epoch+1, np.mean(train_loss), np.mean(val_loss), np.mean(train_acc), np.mean(val_acc)))
val_loss = []
train_loss = []
train_acc = []
val_acc = []


plt.imshow( inp1, origin='lower' )
plt.show()
plt.imshow( inp2, origin='lower' )
plt.show()
plt.imshow( inp3, origin='lower' )
plt.show()
im = imfy(inp3)
img = cv2.GaussianBlur(im, (15, 15), 5)
blobs_log = blob_log(img, min_sigma=6, max_sigma=30, threshold=0.01)
bed_centroids = [Point(i[1], i[0]) for i in blobs_log]
try:
    bed_centroids = get_rects(im, bed_centroids)[0]
except:
    bed_centroids = []
bed_centroids_in = get_mask(bed_centroids, point_s=8)
plt.imshow( bed_centroids_in, origin='lower' )
plt.show()

In [ ]:

def imfy(img):
    return (np.clip(img, 0, 1) * 255).astype(np.uint8)


def get_rects(imgr, centroids):
    bounds = get_mask(centroids, point_s=8).astype(np.uint8)
    img = (imgr).astype(np.uint8)
    #     img[bounds>0] = 0

    img[np.where(bounds == 0)] = 0
    # plt.imshow(img)
    # plt.show()

    shapes = []
    cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for i, c in enumerate(cnts[0]):
        M = cv2.moments(c)
        box = cv2.boundingRect(c);
        x1, y1, w, h = box
        x2, y2 = x1 + w, y1 + h

        ii = imgr[y1:y2, x1:x2]

        intensity = np.mean(ii)

        if not M["m00"]:
            continue
        cx = M["m10"] / M["m00"]
        cy = M["m01"] / M["m00"]
        shapes.append([cx, cy, intensity])

    return [Point(*i[:2]) for i in sorted(shapes, key=lambda x: x[2], reverse=True)]


def get_rects_2(channel, gcenters=True):
    a = channel.copy()

    a[a < 0.9] = 0
    a[a > 1] = 1

    a = (255 * a).astype(np.uint8)

    _, thresh = cv2.threshold(a, 0, 255, cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    final_c = []
    centers = []
    for c in contours:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.01 * perimeter, True)
        x, y, w, h = cv2.boundingRect(approx)
        shapely_box = box(x, y, x + w, y + h)
        final_c.append(shapely_box.buffer(5, join_style=2, cap_style=2))

        mask = np.zeros((256, 256))
        mask = cv2.drawContours(mask, [c], -1, 255, -1)

        rms = channel.copy()
        rms[mask == 0] = 0

        M = cv2.moments(rms)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centers.append(Point(cX, cY))

    if gcenters:
        return centers
    return MultiPolygon(final_c).buffer(0)


In [ ]:
!wget https://kkb-production.jupyter-proxy.kaggle.net/k/135566844/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..WLiAt3FH-rYm5_WhdDdgiw.stF9Kn3XBkHPDCo3PIYGXlPMmEinxV8-_6Dmtc2HVl-q_NUpyY73NAqDex76zk8O1GOPXf9bZn3ES8D1sMIvX73DMwZkrkblZZ0FimPkNQ5C_L3iA5oOPS4uC3NufmJorIfNCBDfqC6S7RmxWKe2hJueQzyDraega-fdiqjIRcTbeuGk-kNbQMP67XzZKFiLga2Mhoeq0StrgDtsQSG74g.JGjOLtbpCgQnxFP9Zqy_ag/proxy/files/bedroom_0.pth

In [ ]:
torch.save(model, '/kaggle/working/bedroom_first_unet_1000.pth')

# pix2Pix


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
from tqdm.auto import tqdm

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/eriklindernoren/PyTorch-GAN/
%cd PyTorch-GAN/implementations/pix2pix

In [ ]:
import os
import numpy as np
import math
import itertools
import time
import datetime
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from models import *
from datasets import *

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
class opt:
    epoch = 0
    n_epochs = 10 #change to 50 for train
    dataset_name = 'test1'
    batch_size = 3
    lr = 0.0002
    b1 = 0.5
    b2 = 0.999
    decay_epoch = 100
    n_cpu = 4
    img_height = 256
    img_width = 256
    channels = 4
    sample_interval = 100
    checkpoint_interval = 1

In [ ]:
os.makedirs("/kaggle/working/images/%s" % opt.dataset_name, exist_ok=True)
os.makedirs("/kaggle/working/saved_models/%s" % opt.dataset_name, exist_ok=True)


In [ ]:
cuda = True if torch.cuda.is_available() else False

# Loss functions
criterion_GAN = torch.nn.MSELoss()
criterion_pixelwise = torch.nn.L1Loss()

# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel = 100

# Calculate output of image discriminator (PatchGAN)
patch = (1, opt.img_height // 2 ** 4, opt.img_width // 2 ** 4)

# Initialize generator and discriminator
generator = GeneratorUNet()
discriminator = Discriminator()
# with open('gen_draw_rooms.pth', 'rb') as f:
#     generator = torch.load(f).cuda()

if cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    criterion_GAN.cuda()
    criterion_pixelwise.cuda()

if opt.epoch != 0:
    # Load pretrained models
    generator.load_state_dict(torch.load("/kaggle/working/saved_models/%s/generator_%d.pth" % (opt.dataset_name, opt.epoch)))
    discriminator.load_state_dict(torch.load("/kaggle/working/saved_models/%s/discriminator_%d.pth" % (opt.dataset_name, opt.epoch)))
else:
    # Initialize weights
    generator.apply(weights_init_normal)
    discriminator.apply(weights_init_normal)

In [ ]:
1e-3

In [ ]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr= 0.0001, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr= 0.0001, betas=(opt.b1, opt.b2))

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
def sample_images(batches_done):
    """Saves a generated sample from the validation set"""
    X, y = next(iter(plan_dataloader_val))
    with torch.no_grad():
        outputs = model(X.cuda()).cpu()
        X = outputs
    real_A = Variable(X.type(Tensor))
    real_B = Variable(y.type(Tensor))
    fake_B = generator(real_A)
    img_sample = torch.cat((real_A.data, fake_B.data, real_B.data), -2).cpu().numpy().astype(np.float32)
    img_sample -=img_sample.min()
    img_sample/=img_sample.max()
    img_sample = img_sample.transpose(0,2,3,1)
    plt.figure(figsize=[10,20])
    for row in range(3):
        plt.subplot(1,3,row+1)
        plt.imshow(img_sample[row])
    plt.show()
    

In [ ]:
discriminator.model[0] = nn.Conv2d(2, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)).cuda()
generator.down1.model[0] = nn.Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)).cuda()

In [ ]:
generator.final[2] = nn.Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1)).cuda()

In [ ]:
train_plans, test_plans = train_test_split(plans, test_size=0.02, shuffle=True, random_state=1997)



train_plans = [i for i in train_plans if i['front'] and i['kitchen'] and i['general']]
test_plans = [i for i in test_plans if i['front'] and i['kitchen'] and i['general']]

plan_dataset = PlanDataset(train_plans)
plan_dataset_val = PlanDataset(test_plans)

plan_dataloader = DataLoader(plan_dataset, batch_size=16, shuffle=True, num_workers=opt.n_cpu)
plan_dataloader_val = DataLoader(plan_dataset_val, batch_size=16, shuffle=True, num_workers=opt.n_cpu)

In [ ]:
### prev_time = time.time()

for epoch in range(opt.epoch, 100):
    for i, (X, y) in enumerate(plan_dataloader):



        # ------------------
        #  Train Generators
        # ------------------

        optimizer_G.zero_grad()
        
        X = X.cuda()
        
        with torch.no_grad():
            outputs = model(X)
            X = outputs
                    
                    
        y = y.cuda()
        
        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((X.size(0), *patch))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((X.size(0), *patch))), requires_grad=False)
        
        # GAN loss
        fake_B = generator(X)
        pred_fake = discriminator(fake_B, X)
        loss_GAN = criterion_GAN(pred_fake, valid)
        # Pixel-wise loss
        loss_pixel = criterion_pixelwise(fake_B, y)

        # Total loss
        loss_G = loss_GAN + lambda_pixel * loss_pixel

        loss_G.backward()

        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Real loss
        pred_real = discriminator(y, X)
        loss_real = criterion_GAN(pred_real, valid)

        # Fake loss
        pred_fake = discriminator(fake_B.detach(), X)
        loss_fake = criterion_GAN(pred_fake, fake)

        # Total loss
        loss_D = 0.5 * (loss_real + loss_fake)

        loss_D.backward()
        optimizer_D.step()

        # --------------
        #  Log Progress
        # --------------

        # Determine approximate time left
        batches_done = epoch * len(plan_dataloader) + i
        batches_left = opt.n_epochs * len(plan_dataloader) - batches_done
        time_left = 0
        prev_time = 0

        # Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, pixel: %f, adv: %f] ETA: %s"
            % (
                epoch,
                opt.n_epochs,
                i,
                len(plan_dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_pixel.item(),
                loss_GAN.item(),
                time_left,
            )
        )

        # If at sample interval save image
        if batches_done % 1000 == 0:
            sample_images(batches_done)

    if opt.checkpoint_interval != -1 and epoch % opt.checkpoint_interval== 0:
        # Save model checkpoints
        torch.save(generator.state_dict(), "/kaggle/working/saved_models/%s/generator_%d.pth" % (opt.dataset_name, epoch))
        torch.save(discriminator.state_dict(), "/kaggle/working/saved_models/%s/discriminator_%d.pth" % (opt.dataset_name, epoch))

In [ ]:
torch.load(generator, 'gen_draw_rooms.pth')

In [ ]:
import requests

files = {
    'file': ('gen_draw_rooms.pth', open('gen_draw_rooms.pth', 'rb')),
}

url = 'https://api.anonfiles.com/upload'
response = requests.post(url, files=files)

data = response.json()

print(data['data']['file']['url']['short'])

# Test

In [ ]:
bed_model = BedCentRegressor().load_from_checkpoint("/kaggle/input/bed-centroids-70ep/xxyy/epoch=20-step=15435.ckpt")

In [ ]:
bath_model = BedCentRegressor().load_from_checkpoint("/kaggle/input/bathroom-more-train/xxyy/epoch=13-step=10290.ckpt")

In [ ]:
kit_model = BedCentRegressor().load_from_checkpoint("/kaggle/input/kitchen-model/kitchen.ckpt")

In [ ]:
d = """
POLYGON ((139.8806629834254 195.8895027624309, 139.8806629834254 200.4154696132597, 157.9845303867403 200.4154696132597, 157.9845303867403 195.8895027624309, 139.8806629834254 195.8895027624309))
POLYGON ((26.73149171270718 76.33964778841968, 25.59999999999999 77.47113950112686, 25.59999999999999 107.2448825983206, 26.73149171270718 108.3763743110278, 26.73149171270718 130.2629834254143, 49.36132596685083 130.2629834254143, 49.36132596685083 199.2839779005525, 230.4 199.2839779005525, 230.4 56.71602209944751, 26.73149171270718 56.71602209944751, 26.73149171270718 76.33964778841968))
"""

In [ ]:
import re

pattern = r'<POINT \((\d+(?:\.\d+)?)\s(\d+(?:\.\d+)?)\)>'

def parse_points(input_str):
    coords_str = re.findall(pattern, input_str)
    points = [Point(float(x), float(y)) for x, y in coords_str]
    return points


d = d.split("\n")
# points = [parse_points(i) for i in d if "POINT" in i]
polygon = [shapely.from_wkt(i) for i in d if "POLYGON" in i]

inner_poly = max(polygon, key=lambda x: x.area)
door_poly = min(polygon, key=lambda x: x.area)
# bedrooms, bathrooms =  points

In [ ]:
model.eval();

In [ ]:

import numpy as np
from scipy.signal import convolve2d
from skimage.feature import blob_dog, blob_log, blob_doh


def get_rects(imgr, centroids, door, get_max=0):
    bounds = get_mask(centroids, point_s=8).astype(np.uint8)
    d = ~get_mask(door.centroid, point_s=30).astype(np.uint8)
#     imgr[:, :, 0] = (imgr[:, :, 0] - d).astype(np.uint8)
#     plt.imshow(imgr)
#     plt.gca().invert_yaxis() 
#     plt.show()
    img = (imgr).astype(np.uint8)
    img[np.where(bounds == 0)] = 0

    if get_max:
        kernel = np.ones((8, 8))
        convolved = convolve2d(imgr.reshape((256, 256)), kernel, mode='valid')
        plt.imshow(convolved)
        plt.show()
        y, x = np.unravel_index(np.argmax(convolved), convolved.shape)
        return [Point(x, y)]

    shapes = []
    cnts = cv2.findContours(imgr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    for i, c in enumerate(cnts[0]):
        M = cv2.moments(c)
        box = cv2.boundingRect(c);
        x1, y1, w, h = box
        x2, y2 = x1 + w, y1 + h

        ii = img[y1:y2, x1:x2]
        
        ii = ii * (ii >= np.median(imgr))
        intensity = np.sum(ii)

        if not M["m00"]:
            continue
        cx = M["m10"] / M["m00"]
        cy = M["m01"] / M["m00"]
        shapes.append([cx, cy, intensity])

    return [Point(*i[:2]) for i in sorted(shapes, key=lambda x: x[2], reverse=True)]

In [ ]:
# bed_model.eval();
# bath_model.eval();
# kit_model.eval();
# bed_model.train();
# bath_model.train();
# kit_model.train();


In [ ]:
from shapely.geometry import LineString
neigh = LineString([(49.36132596685083, 199.2839779005525),
 (230.4, 199.2839779005525)]).union(LineString([ (230.4, 56.71602209944751),
 (26.73149171270718, 56.71602209944751)])).buffer(1, join_style=2)

In [ ]:
gpd.GeoSeries([inner_poly, door_poly, neigh]).plot(cmap="Dark2_r")

In [ ]:
# bedrooms = []
# bathrooms = []



# no_bedrooms = 3
# no_bathrooms = 3

# area = 180.0
# area = min(area / 400, 1)

# for i in range(no_bedrooms):
#     x = np.zeros((2, 256, 256 , 9))

#     x[0,:,:,0] = get_mask(door_poly.centroid, (256, 256), point_s=5) > 0
#     x[0,:,:,1] = get_mask(inner_poly, (256, 256), point_s=5) > 0

#     length_box = 256*area/2
#     x[0,:,:,2] = get_mask(box(128-length_box, 128-length_box, 128+length_box, 128+length_box), (256, 256), point_s=10) > 0        
#     x[0,:,:,3] = get_mask([neigh], (256, 256), point_s=5) > 0
#     x[0,:,:,4] = get_mask(bedrooms, (256, 256), point_s=5) > 0
#     x[0,:,:,4 + no_bedrooms] = 1.0

#     out = bed_model(torch.from_numpy(x).permute((0, 3, 1, 2)).cuda().float())
#     out = out.detach().cpu()[0][0]

#     im = out.numpy()
#     current = get_mask(bedrooms, point_s=10)
#     im[current!=0] = 0
#     img = cv2.GaussianBlur(im, (15, 15), 5)
# #     plt.imshow(img)
# #     plt.show()
#     blobs_log = blob_log(img, min_sigma=4, max_sigma=30, threshold=0.01)
#     bed_centroids = [Point(i[1], i[0]) for i in blobs_log]
#     centroids = get_rects(im, bed_centroids, door_poly, get_max=True)
#     bedrooms.extend(centroids)

    
# for i in range(no_bathrooms):
#     x = np.zeros((2, 256, 256 , 9))

#     x[0,:,:,0] = get_mask(door_poly.centroid, (256, 256), point_s=5) > 0
#     x[0,:,:,1] = get_mask(inner_poly, (256, 256), point_s=5) > 0

#     length_box = 256*area/2
#     x[0,:,:,2] = get_mask(box(128-length_box, 128-length_box, 128+length_box, 128+length_box), (256, 256), point_s=10) > 0        
#     x[0,:,:,3] = get_mask([neigh], (256, 256), point_s=5) > 0
#     x[0,:,:,4] = get_mask(bedrooms, (256, 256), point_s=5) > 0
#     x[0,:,:,5] = get_mask(bathrooms, (256, 256), point_s=5) > 0
#     x[0,:,:,4 + no_bathrooms] = 1.0

#     out = bath_model(torch.from_numpy(x).permute((0, 3, 1, 2)).cuda().float())
#     out = out.detach().cpu()[0][0]

# #     plt.imshow(out)
# #     plt.show()

#     im = out.numpy()
#     current = get_mask(bedrooms + bathrooms, point_s=10)
#     im[current!=0] = 0
#     img = cv2.GaussianBlur(im, (15, 15), 5)
# #     plt.imshow(img)
# #     plt.show()
#     blobs_log = blob_log(img, min_sigma=4, max_sigma=30, threshold=0.01)
#     bath_centroids = [Point(i[1], i[0]) for i in blobs_log]
#     centroids = get_rects(im, bath_centroids, door_poly, get_max=True)
#     bathrooms.extend(centroids)
    

# x = np.zeros((2, 256, 256 , 9))

# x[0,:,:,0] = get_mask(door_poly.centroid, (256, 256), point_s=5) > 0
# x[0,:,:,1] = get_mask(inner_poly, (256, 256), point_s=5) > 0

# length_box = 256*area/2
# x[0,:,:,2] = get_mask(box(128-length_box, 128-length_box, 128+length_box, 128+length_box), (256, 256), point_s=10) > 0        
# x[0,:,:,3] = get_mask([neigh], (256, 256), point_s=5) > 0
# x[0,:,:,4] = get_mask(bedrooms, (256, 256), point_s=5) > 0
# x[0,:,:,5] = get_mask(bathrooms, (256, 256), point_s=5) > 0

# out = kit_model(torch.from_numpy(x).permute((0, 3, 1, 2)).cuda().float())
# out = out.detach().cpu()[0][0]

# # plt.imshow(out)
# # plt.show()

# im = out.numpy()
# current = get_mask(bedrooms + bathrooms, point_s=10)
# im[current!=0] = 0
# img = cv2.GaussianBlur(im, (15, 15), 5)
# plt.imshow(img)
# plt.show()
# blobs_log = blob_log(img, min_sigma=4, max_sigma=30, threshold=0.01)
# kit_centroids = [Point(i[1], i[0]) for i in blobs_log]
# centroids = get_rects(im, kit_centroids, door_poly, get_max=True)
# kitchen = centroids[0]


In [ ]:
# aio_model = BedCentRegressor().load_from_checkpoint("/kaggle/working/xxyy/epoch=20-step=14028.ckpt")

In [ ]:
# model = model.cuda()

In [ ]:

def get_rects(imgr, centroids, door, get_max=0):
    bounds = get_mask(centroids, point_s=8).astype(np.uint8)
    d = ~get_mask(door.centroid, point_s=30).astype(np.uint8)
    img = (imgr).astype(np.uint8)
    img[np.where(bounds == 0)] = 0

    if get_max:
        kernel = np.ones((1, 1))
        convolved = convolve2d(imgr.reshape((256, 256)), kernel, mode='same', boundary='fill', fillvalue=0)
#         plt.imshow(imgr)
#         plt.show()
        y, x = np.unravel_index(np.argmax(convolved), convolved.shape)
        return [Point(x, y)]
    
    
def get_rects(imgr, centroids, door, get_max=0):
    bounds = get_mask(centroids, point_s=8)
#     d = ~get_mask(door.centroid, point_s=6).astype(np.uint8)
#     imgr[:, :, 0] = (imgr[:, :, 0] - d).astype(np.uint8)
#     plt.imshow(imgr)
#     plt.gca().invert_yaxis() 
#     plt.show()
#     imgr = abs((imgr*bounds*255)).astype(np.uint8)
    
    imgr[np.where(bounds == 0)] = 0
    
    kernel = np.ones((2, 2))
    convolved = convolve2d(imgr.reshape((256, 256)), kernel, mode='valid')
#     convolved = convolved - np.min(convolved)
#     convolved = (convolved * 255 / np.max(convolved)).astype(np.uint8)
     
    blobs_log = blob_log(convolved, min_sigma=3, max_sigma=30, threshold=0.01)
#     plt.imshow(convolved)
#     plt.show()
    try:
        p = max(blobs_log, key=lambda c: convolved[int(c[1]), int(c[0])])[:2]
        return [Point(*p[::-1])]
    
    except:
        
        return []
    
    
    
    
    
    if get_max:
        y, x = np.unravel_index(np.argmax(convolved), convolved.shape)
        return [Point(x, y)]
    shapes = []
    cnts = cv2.findContours(convolved, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for i, c in enumerate(cnts[0]):
        M = cv2.moments(c)
        box = cv2.boundingRect(c);
        x1, y1, w, h = box
        x2, y2 = x1 + w, y1 + h

        ii = convolved[y1:y2, x1:x2]

#         ii = ii * (ii >= np.median(convolved))
        
        intensity = np.median(ii)

        if not M["m00"]:
            continue
        cx = M["m10"] / M["m00"]
        cy = M["m01"] / M["m00"]
        
        shapes.append([cx, cy, intensity])
    
    shapes = [i[:2] for i in sorted(shapes, key=lambda x: x[2], reverse=True)]
    if not shapes:
        return []
    
    return [Point(*shapes[0])]

In [ ]:
model.cpu().eval();

In [ ]:
bedrooms = []
bathrooms = []
kitchen = []


no_bedrooms = 3
no_bathrooms = 2
area = 130.0

area = min(area / 400, 1)

x = np.zeros((1, 256, 256 , 12))


x[0,:,:,0] = get_mask(door_poly.centroid, (256, 256), point_s=5) > 0
x[0,:,:,1] = get_mask(inner_poly, (256, 256), point_s=5) > 0

length_box = 256*area/2
x[0,:,:,2] = get_mask(box(128-length_box, 128-length_box, 128+length_box, 128+length_box), (256, 256), point_s=10) > 0        
x[0,:,:,3] = get_mask([neigh], (256, 256), point_s=5) > 0
x[0,:,:,3 + no_bedrooms] = 1.0
x[0,:,:,7 + no_bathrooms] = 1.0



with torch.no_grad():
    x = torch.from_numpy(x).permute((0, 3, 1, 2)).float()

    bottle_neck = model.forward_encoder_main(x)


    for i in range(no_bedrooms + no_bathrooms + 1) :

        x_task = np.zeros((1, 256, 256 , 3))

        x_task[0,:,:,0] = get_mask(bedrooms, (256, 256), point_s=5) > 0
        x_task[0,:,:,1] = get_mask(bathrooms, (256, 256), point_s=5) > 0
        x_task[0,:,:,2] = get_mask(kitchen, (256, 256), point_s=5) > 0

        if len(bedrooms) < no_bedrooms:
    #         x_task[0,:,:,3] = 1.0
            state = 1
        elif len(bathrooms) < no_bathrooms:
    #         x_task[0,:,:,4] = 1.0
            state = 2
        else:
    #         x_task[0,:,:,5] = 1.0
            state = 3

        x_task = torch.from_numpy(x_task).permute((0, 3, 1, 2)).float()

        other_features = model.forward_encoder_mini(x_task)


        out = model.forward_decoder(bottle_neck, other_features, state=state)

        out = out.detach().cpu()[0][0]

        im = out.numpy()
        current = get_mask(bedrooms + bathrooms, point_s=10)
        im[current!=0] = 0
        img = cv2.GaussianBlur(im, (15, 15), 5)
#         plt.imshow(img)
#         plt.show()
        blobs_log = blob_log(img, min_sigma=4, max_sigma=30, threshold=0.01)
        bed_centroids = [Point(i[1], i[0]) for i in blobs_log]
        
        centroids = get_rects(im, bed_centroids, door_poly, get_max=0)
        print(centroids)
        if len(bedrooms) < no_bedrooms:
            bedrooms.extend(centroids)
        elif len(bathrooms) < no_bathrooms:
            bathrooms.extend(centroids)
        else:
            kitchen = centroids


In [ ]:
print(bedrooms)
print(bathrooms)
print([kitchen])
print(door_poly)
print(inner_poly)

In [ ]:
gpd.GeoSeries([inner_poly, door_poly] + [unary_union(bedrooms), unary_union(bathrooms), kitchen[0]]).plot(cmap="Dark2_r")

In [ ]:
plt.imshow(x[0, :, :, 1])

In [ ]:
trainer.save_checkpoint("out.ckpt")